# Importazione librerie

In [1]:
import random
import numpy as np
from deap import base, creator, tools
import pandas as pd
import matplotlib.pyplot as plt
import warnings

In [2]:
warnings.simplefilter(action='ignore', category=FutureWarning)

# Sopprimi i RuntimeWarning
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Lettura/generazione dati

In [3]:
dati = pd.read_csv('Bitcoin (€) da yfinance dal 17-09-2014 al 24-04-2024.csv')
dati = dati[['Date', 'Close']]
dati = dati.rename(columns = {'Date':'Timestamp', 'Close': 'Price'})
dati['Timestamp'] = pd.to_datetime(dati['Timestamp'])
dati['Timestamp'] = dati['Timestamp'].dt.strftime('%Y-%m-%d')
dati['Timestamp'] = pd.to_datetime(dati['Timestamp'], format='%Y-%m-%d')
dati

,Timestamp,Price
0,2014-09-17,355.957367
1,2014-09-18,328.539368
2,2014-09-19,307.761139
3,2014-09-20,318.758972
4,2014-09-21,310.632446
...,...,...
3503,2024-04-20,59876.710938
3504,2024-04-21,60956.074219
3505,2024-04-22,60919.242188
3506,2024-04-23,62729.296875


In [4]:
data_inizio = '2021-02-01'
data_fine = '2021-07-01'
indice_inizio = dati[dati['Timestamp'] == data_inizio].index[0]
indice_fine = dati[dati['Timestamp'] == data_fine].index[0]
dati = dati[indice_inizio:indice_fine + 1].reset_index(drop=True)
dati

,Timestamp,Price
0,2021-02-01,27790.582031
1,2021-02-02,29501.521484
2,2021-02-03,31119.447266
3,2021-02-04,30859.554688
4,2021-02-05,31658.248047
...,...,...
146,2021-06-27,29013.742188
147,2021-06-28,28866.302734
148,2021-06-29,30139.980469
149,2021-06-30,29555.054688


In [5]:
def calcola_rendimento_portafoglio_ideale(lista_prezzi, capitale_iniziale, bitcoin_iniziali, min_acquisto=0, min_vendita=0, perc_commissione_acquisto=0, perc_commissione_vendita=0):
    capitale = capitale_iniziale
    bitcoin = bitcoin_iniziali
    comprato = False

    for i in range(len(lista_prezzi) - 1):
        # Compra al minimo
        if lista_prezzi[i] < lista_prezzi[i + 1] and not comprato and capitale >= min_acquisto:
            bitcoin_acquistati = (capitale * (1 - perc_commissione_acquisto)) / lista_prezzi[i]
            bitcoin += bitcoin_acquistati
            capitale = 0
            comprato = True
            # print(f'Bitcoin posseduti: {bitcoin}')
        # Vende al massimo
        elif lista_prezzi[i] > lista_prezzi[i + 1] and comprato and bitcoin * lista_prezzi[i] >= min_vendita:
            capitale_vendita = bitcoin * lista_prezzi[i] * (1 - perc_commissione_vendita)
            capitale += capitale_vendita
            bitcoin = 0
            comprato = False
            # print(f'Capitale posseduto: {capitale}')

    # Vende i bitcoin rimasti all'ultimo prezzo
    if comprato and bitcoin * lista_prezzi.iloc[-1] >= min_vendita:
        capitale += bitcoin * lista_prezzi.iloc[-1] * (1 - perc_commissione_vendita)
        bitcoin = 0
        # print(f'Capitale posseduto: {capitale}')
        # print(f'Bitcoin posseduti: {bitcoin}')

    valore_iniziale = capitale_iniziale + bitcoin_iniziali * lista_prezzi[0]
    valore_finale = capitale + bitcoin * lista_prezzi.iloc[-1]

    rendimento = valore_finale / valore_iniziale

    return [rendimento, valore_finale]

In [6]:
def estrai_periodi_casuali(dati, N, L, start_date, end_date, random_state):
    # Filtra i dati tra start_date e end_date
    dati_filtrati = dati[(dati['Timestamp'] >= pd.to_datetime(start_date)) & 
                         (dati['Timestamp'] <= pd.to_datetime(end_date))]
    
    # Calcola il numero massimo di periodi non sovrapposti
    num_max_periodi = len(dati_filtrati) - L + 1
    
    # Controlla se è possibile estrarre N periodi diversi
    if num_max_periodi < N:
        raise ValueError(f"Non è possibile estrarre {N} periodi diversi di lunghezza {L} tra le date {start_date} e {end_date}.")
    
    # Inizializza una lista per i periodi estratti e un set per memorizzare gli estremi
    periodi_estratti = []
    estremi_estratti = set()
    
    # Inizializza il generatore di numeri casuali con il seed dato
    rng = np.random.RandomState(random_state)
    
    while len(periodi_estratti) < N:
        # Seleziona un indice casuale per l'inizio del periodo
        indice_inizio = rng.randint(0, len(dati_filtrati) - L + 1)
        
        # Determina gli estremi del periodo
        estremo_inizio = dati_filtrati.iloc[indice_inizio]['Timestamp']
        estremo_fine = dati_filtrati.iloc[indice_inizio + L - 1]['Timestamp']
        
        # Controlla se gli estremi sono già stati utilizzati
        if (estremo_inizio, estremo_fine) not in estremi_estratti:
            # Aggiungi gli estremi al set
            estremi_estratti.add((estremo_inizio, estremo_fine))
            
            # Estrai il periodo e aggiungilo alla lista dei periodi estratti
            periodo = dati_filtrati.iloc[indice_inizio:indice_inizio + L]

            periodo = periodo.reset_index(drop=True)

            periodi_estratti.append(periodo)
    
    return periodi_estratti

In [7]:
# Esempio di utilizzo:
start_date = dati['Timestamp'].iloc[0]
end_date = dati['Timestamp'].iloc[-1]
N = 100 # Numero di periodi da estrarre
L = 60 # Lunghezza di ogni periodo in giorni
random_state = 8

In [8]:
# periodi_casuali_esistenti = estrai_periodi_casuali(dati, N, L, start_date, end_date, random_state)

In [9]:
# datasets = [periodi_casuali_esistenti[i]['Price'].values for i in range(len(periodi_casuali_esistenti))]

# lunghezza_dataset = len(periodi_casuali_esistenti[0])

# # Converte i dataset generati in DataFrame
# generated_dfs = [pd.DataFrame({'Timestamp': dati['Timestamp'].iloc[:lunghezza_dataset], 'Price': prices}) for prices in datasets] # Fingo che le date siano tutte iniziate con data_inizio

# # Plotting
# plt.figure(figsize=(14, 7))

# # Plot del dataset originale
# plt.plot(dati['Timestamp'].iloc[:lunghezza_dataset], dati['Price'].iloc[:lunghezza_dataset].values, label='Original', linewidth=3)

# # Plot dei dataset generati
# for i, df in enumerate(generated_dfs):
#     plt.plot(df['Timestamp'], df['Price'], label=f'Generated Dataset {i+1}', alpha=0.7)

# plt.title('Generated Bitcoin Price Datasets')
# plt.xlabel('Time')
# plt.ylabel('Price')
# # plt.yscale('log')
# plt.grid()
# # plt.legend()
# plt.show()

# Scelta dei dati da usare

In [10]:
# dati_da_usare = periodi_casuali_esistenti.copy()

In [11]:
# # Step 1: Leggi il dataset
# dati_da_usare = pd.read_csv('7 cluster con 100 dataset sintetici\Cluster_0.csv')

# # Step 2: Inizializza una lista vuota per memorizzare i dataframes
# lista_dataframes = []

# # Step 3: Estrai i nomi delle colonne che contengono i prezzi (ipotizzando che tu abbia una colonna 'Timestamp' e 100 colonne di prezzo)
# for colonna in dati_da_usare.columns[1:len(dati_da_usare.iloc[:, 0]) + 1]:  # Saltiamo la prima colonna ('Timestamp') e prendiamo le 100 colonne successive
#     # Step 4: Crea un dataframe con le due colonne 'Timestamp' e l'attuale colonna 'Price'
#     df_temp = dati_da_usare[['Timestamp', colonna]].copy()
#     df_temp.columns = ['Timestamp', 'Price']  # Rinomina la colonna del prezzo come 'Price'
    
#     # Step 5: Aggiungi il dataframe alla lista
#     lista_dataframes.append(df_temp)

# dati_da_usare = lista_dataframes.copy()

In [12]:
dati_da_usare = [dati]

In [13]:
print('Fattore di ritorno medio dei prezzi estratti:', np.mean([df['Price'].iloc[-1] / df['Price'].iloc[0] for df in dati_da_usare]))

ideal_returns = [calcola_rendimento_portafoglio_ideale(df['Price'], 1000, 0, 5, 5, 0.001, 0.001)[0] for df in dati_da_usare]

print('Media dei fattori di rendimento massimi ottenibili:', np.mean(ideal_returns))

perc_suff = np.mean([1/calcola_rendimento_portafoglio_ideale(df['Price'], 1000, 0, 5, 5, 0.001, 0.001)[0] for df in dati_da_usare])

print(f'Punteggio percentuale considerato "sufficiente": {round(perc_suff*100,2)}%')

Fattore di ritorno medio dei prezzi estratti: 1.0198208166576217
Media dei fattori di rendimento massimi ottenibili: 12.487911471773899
Punteggio percentuale considerato "sufficiente": 8.01%


In [14]:
dati_da_usare[0]

,Timestamp,Price
0,2021-02-01,27790.582031
1,2021-02-02,29501.521484
2,2021-02-03,31119.447266
3,2021-02-04,30859.554688
4,2021-02-05,31658.248047
...,...,...
146,2021-06-27,29013.742188
147,2021-06-28,28866.302734
148,2021-06-29,30139.980469
149,2021-06-30,29555.054688


# Strategie

## Trading a % senza micro variazioni

In [15]:
# # Funzione per calcolare l'ultimo prezzo di riferimento discesa
# def calcola_prezzo_riferimento_discesa(dati, soglia_discesa):
#     variazione_cumulata = 1
#     cont = 0

#     for j in range(len(dati), 0, -1):
#         variazione_giorno_precedente = dati['Variazione'].iloc[j-1]

#         if pd.isna(variazione_giorno_precedente):
#             continue

#         if variazione_giorno_precedente < 1:  # Prezzo in discesa
#             variazione_cumulata = 1
#             cont = 0
#         else:
#             variazione_cumulata *= variazione_giorno_precedente
#             if variazione_cumulata != variazione_giorno_precedente:
#                 cont += 1
#             if variazione_cumulata - 1 >= soglia_discesa:
#                 # Se la variazione supera la soglia, restituisci il riferimento
#                 if j + cont - 1 == len(dati) - 1:
#                     # Se il riferimento è l'ultimo giorno, restituisci None
#                     return None, None
#                 return dati['Timestamp'].iloc[j+cont-1], dati['Price'].iloc[j+cont-1]

#     # Se non si è trovata una variazione che supera la soglia, ritorna il primo giorno
#     return dati['Timestamp'].iloc[0], dati['Price'].iloc[0]

# # Funzione per calcolare l'ultimo prezzo di riferimento salita
# def calcola_prezzo_riferimento_salita(dati, soglia_salita):
#     variazione_cumulata = 1
#     cont = 0

#     for j in range(len(dati), 0, -1):
#         variazione_giorno_precedente = dati['Variazione'].iloc[j-1]

#         if pd.isna(variazione_giorno_precedente):
#             continue

#         if variazione_giorno_precedente > 1:  # Prezzo in salita
#             variazione_cumulata = 1
#             cont = 0
#         else:
#             variazione_cumulata *= variazione_giorno_precedente
#             if variazione_cumulata != variazione_giorno_precedente:
#                 cont += 1
#             if 1 - variazione_cumulata >= soglia_salita:
#                 # Se la variazione supera la soglia o il prezzo di riferimento è superiore al prezzo attuale, restituisci il riferimento
#                 if j + cont - 1 == len(dati) - 1:
#                     # Se il riferimento è l'ultimo giorno, restituisci None
#                     return None, None
#                 return dati['Timestamp'].iloc[j+cont-1], dati['Price'].iloc[j+cont-1]

#     # Se non si è trovata una variazione che supera la soglia, ritorna il primo giorno
#     return dati['Timestamp'].iloc[0], dati['Price'].iloc[0]

# def calcola_percentuale(variazione, coefficienti, A, B):
#     if isinstance(coefficienti, list):
#         # Se coefficienti è una lista, esegue la somma ponderata delle variazioni
#         return min(max(sum([c * variazione ** i for i, c in enumerate(coefficienti)]), 0), 1)
    
#     elif isinstance(coefficienti, str) and coefficienti != 'custom_exp':
#         # Se coefficienti è una stringa, interpreta la stringa come una formula matematica
#         # Sostituisci 'x' con il valore di variazione
#         formula = coefficienti.replace('x', str(variazione))
#         # Usa eval per calcolare il risultato
#         try:
#             risultato = eval(formula)
#         except Exception as e:
#             raise ValueError(f"Errore nell'interpretazione della formula: {e}")
#         return min(max(risultato, 0), 1)
    
#     elif coefficienti == 'custom_exp':
#         return eval(f'variazione ** {A} / ({B} + (1 - {B}) * variazione ** {A})')
    
#     else:
#         raise TypeError("Il parametro 'coefficienti' deve essere una lista, una stringa o 'custom_exp'.")

In [16]:
# def trading_perc_senza_micro_variazioni_velocizzato(dati, inverti_riferimento_acquisto, inverti_riferimento_vendita, acquisto_rispetto_liquidità_iniziale, vendita_rispetto_massimo_btc_posseduti,
#      compra_pure, vendi_pure, soglia_vendita, A_a, B_a, A_v, B_v, coefficienti_acquisto, coefficienti_vendita,
#      compra_solo_in_discesa, vendi_solo_in_salita, soglia_acquisto_rispetto_riferimento,
#      soglia_vendita_rispetto_riferimento, soglia_calcolo_variazioni_discesa, soglia_calcolo_variazioni_salita, perc_relativa, finestra_minimo, finestra_massimo, nuovi_min_all_in, nuovi_max_all_in, soglia_acquisti_vicini, soglia_vendite_vicine, liquidità_iniziale, bitcoin_iniziali):
    

#     # Parametri iniziali
#     liquidità = liquidità_iniziale
#     bitcoin = bitcoin_iniziali
#     valore_totale_portafoglio = liquidità + bitcoin * dati['Price'][0]

#     # Calcolo variazioni giornaliere
#     dati['Variazione'] = dati['Price'] / dati['Price'].shift(1)

#     # Inizializza la lista delle transazioni
#     transazioni = []

#     # Variabili per tenere traccia degli acquisti consecutivi
#     acquisti_consecutivi = []
#     quantità_euro_totale = 0  # Somma totale degli euro spesi negli acquisti consecutivi

#     liquidità_iniziale_in_btc = liquidità_iniziale / dati['Price'].iloc[0]

#     ultimo_prezzo_acquisto = None
#     ultimo_prezzo_vendita = None
#     ultima_azione = None
#     azione = None


#     # Simulazione del trading giorno per giorno
#     for i in range(1, len(dati)):
#         data_odierna = dati['Timestamp'].iloc[i]
#         prezzo_attuale = dati['Price'].iloc[i]
#         variazione_giornaliera_prezzo_percentuale = (dati['Variazione'].iloc[i] - 1) * 100
#         if perc_relativa:
#             if finestra_minimo <= i:
#                 minimo_ultimi_giorni = np.min(dati['Price'].iloc[i-finestra_minimo:i+1])

#             else:
#                 minimo_ultimi_giorni = np.min(dati['Price'].iloc[:i+1])

#             if finestra_massimo <= i:
#                 massimo_ultimi_giorni = np.max(dati['Price'].iloc[i-finestra_massimo:i+1])

#             else:
#                 massimo_ultimi_giorni = np.max(dati['Price'].iloc[:i+1])
        
#         # Calcola i prezzi di riferimento discesa e salita
#         timestamp_riferimento_discesa, prezzo_riferimento_discesa = calcola_prezzo_riferimento_discesa(dati.iloc[:i+1], soglia_calcolo_variazioni_discesa)
#         timestamp_riferimento_salita, prezzo_riferimento_salita = calcola_prezzo_riferimento_salita(dati.iloc[:i+1], soglia_calcolo_variazioni_salita)
#         # timestamp_riferimento_discesa, prezzo_riferimento_discesa = rif_discesa_sbagliato(dati.iloc[:i], soglia_calcolo_variazioni_discesa)
#         # timestamp_riferimento_salita, prezzo_riferimento_salita = rif_salita_sbagliato(dati.iloc[:i], soglia_calcolo_variazioni_salita)

#         # Scegli il prezzo di riferimento più recente (se esiste)
#         prezzo_riferimento = None
#         if prezzo_riferimento_discesa is not None and prezzo_riferimento_salita is not None:
#             if timestamp_riferimento_discesa > timestamp_riferimento_salita:
#                 prezzo_riferimento = prezzo_riferimento_discesa
#                 tipo_riferimento = 'discesa'
#             else:
#                 prezzo_riferimento = prezzo_riferimento_salita
#                 tipo_riferimento = 'salita'
#         elif prezzo_riferimento_discesa is not None:
#             prezzo_riferimento = prezzo_riferimento_discesa
#             tipo_riferimento = 'discesa'
#         elif prezzo_riferimento_salita is not None:
#             prezzo_riferimento = prezzo_riferimento_salita
#             tipo_riferimento = 'salita'

#         # Se nessun riferimento è disponibile, passa al giorno successivo
#         if prezzo_riferimento is None:
#             continue

#         # Variabili per tracciare l'azione e la transazione
#         azione = '-'
#         # percentuale_liquidità_usata = 0
#         # percentuale_bitcoin_usata = 0
#         # percentuale_liquidità_usata_rispetto_rif = 0
#         # percentuale_bitcoin_usata_rispetto_rif = 0
#         # euro_spesi_o_ricavati = 0
#         # bitcoin_acquistati_o_venduti = 0

#         # Logica di acquisto o vendita in base al riferimento più recente
#         if tipo_riferimento == 'discesa':  # Logica di acquisto
#             if not perc_relativa:
#                 if inverti_riferimento_acquisto:
#                     variazione_percentuale = (prezzo_riferimento_discesa - prezzo_attuale) / prezzo_attuale
#                 else:
#                     variazione_percentuale = (prezzo_riferimento_discesa - prezzo_attuale) / prezzo_riferimento_discesa
#             else:
#                 den = prezzo_riferimento_discesa - minimo_ultimi_giorni
#                 if den > 0: # Aggiungo questo controllo perchè in generale potrebbe anche accadere che il minimo sia superiore all'ultimo prezzo_riferimento_discesa salvato
#                     num = prezzo_riferimento_discesa - prezzo_attuale
#                     variazione_percentuale = num / den
#                     if not nuovi_min_all_in and variazione_percentuale == 1:
#                         if inverti_riferimento_acquisto:
#                             variazione_percentuale = (prezzo_riferimento_discesa - prezzo_attuale) / prezzo_attuale
#                         else:
#                             variazione_percentuale = (prezzo_riferimento_discesa - prezzo_attuale) / prezzo_riferimento_discesa
#                 else:
#                     variazione_percentuale = 0


#             percentuale_acquisto = calcola_percentuale(abs(variazione_percentuale), coefficienti_acquisto, A_a, B_a)
#             liquidità_da_usare = liquidità_iniziale if acquisto_rispetto_liquidità_iniziale else liquidità
#             euro_da_spendere = liquidità_da_usare * percentuale_acquisto

#             if euro_da_spendere > liquidità:
#                 if compra_pure:
#                     euro_da_spendere = liquidità  # Usa tutto
#                 else:
#                     euro_da_spendere = 0  # Non fare niente

#             if euro_da_spendere > 0 and (compra_solo_in_discesa and variazione_giornaliera_prezzo_percentuale < 0 or not compra_solo_in_discesa) and prezzo_attuale < prezzo_riferimento_discesa * (1 - soglia_acquisto_rispetto_riferimento) and (soglia_acquisti_vicini > 0 and ultimo_prezzo_acquisto is not None and ultima_azione == 'Acquisto' and abs(prezzo_attuale - ultimo_prezzo_acquisto) >= soglia_acquisti_vicini * ultimo_prezzo_acquisto or soglia_acquisti_vicini == 0 or ultima_azione is None):
#                 # percentuale_liquidità_usata = euro_da_spendere / liquidità
#                 bitcoin_acquistati = euro_da_spendere / prezzo_attuale
#                 liquidità -= euro_da_spendere
#                 bitcoin += bitcoin_acquistati
#                 azione = 'Acquisto'
#                 ultima_azione = azione
#                 # percentuale_liquidità_usata_rispetto_rif = percentuale_acquisto
#                 # euro_spesi_o_ricavati = euro_da_spendere
#                 # bitcoin_acquistati_o_venduti = bitcoin_acquistati
                
#                 # Aggiungi questo acquisto alla lista degli acquisti consecutivi
#                 acquisti_consecutivi.append((prezzo_attuale, euro_da_spendere))
#                 quantità_euro_totale += euro_da_spendere
#                 ultimo_prezzo_acquisto = prezzo_attuale

#         elif tipo_riferimento == 'salita':  # Logica di vendita
#             if not perc_relativa:
#                 if inverti_riferimento_vendita:
#                     variazione_percentuale = (prezzo_attuale - prezzo_riferimento_salita) / prezzo_attuale
#                 else:
#                     variazione_percentuale = (prezzo_attuale - prezzo_riferimento_salita) / prezzo_riferimento_salita
#             else:
#                 den = massimo_ultimi_giorni - prezzo_riferimento_salita
#                 if den > 0: # Aggiungo questo controllo perchè in generale potrebbe anche accadere che il massimo sia inferiore all'ultimo prezzo_riferimento_salita salvato
#                     num = prezzo_attuale - prezzo_riferimento_salita
#                     variazione_percentuale = num / den
#                     if not nuovi_max_all_in and variazione_percentuale == 1:
#                         if inverti_riferimento_vendita:
#                             variazione_percentuale = (prezzo_attuale - prezzo_riferimento_salita) / prezzo_attuale
#                         else:
#                             variazione_percentuale = (prezzo_attuale - prezzo_riferimento_salita) / prezzo_riferimento_salita
#                 else:
#                     variazione_percentuale = 0

#             percentuale_vendita = calcola_percentuale(abs(variazione_percentuale), coefficienti_vendita, A_v, B_v)
#             bitcoin_da_usare = liquidità_iniziale_in_btc if vendita_rispetto_massimo_btc_posseduti else bitcoin
#             bitcoin_da_vendere = bitcoin_da_usare * percentuale_vendita


#             # Controllo del prezzo di vendita rispetto alla media pesata degli acquisti consecutivi
#             if acquisti_consecutivi:
#                 # Calcolo della media pesata dei prezzi di acquisto
#                 somma_pesata = sum(prezzo * euro for prezzo, euro in acquisti_consecutivi)
#                 media_pesata_acquisti = somma_pesata / quantità_euro_totale

#                 # Verifica che il prezzo attuale sia superiore alla soglia rispetto alla media pesata
#                 if prezzo_attuale <= media_pesata_acquisti * (1 + soglia_vendita):
#                     # Se non soddisfa la soglia di vendita, non fare niente
#                     bitcoin_da_vendere = 0

#             if bitcoin_da_vendere > bitcoin:
#                 if vendi_pure:
#                     bitcoin_da_vendere = bitcoin  # Vendi tutto
#                 else:
#                     bitcoin_da_vendere = 0  # Non fare niente

#             if bitcoin_da_vendere > 0 and (vendi_solo_in_salita and variazione_giornaliera_prezzo_percentuale > 0 or not vendi_solo_in_salita) and prezzo_attuale > prezzo_riferimento_salita * (1 + soglia_vendita_rispetto_riferimento) and (soglia_vendite_vicine > 0 and ultimo_prezzo_vendita is not None and ultima_azione == 'Vendita' and abs(prezzo_attuale - ultimo_prezzo_vendita) >= soglia_vendite_vicine * ultimo_prezzo_vendita or soglia_vendite_vicine == 0 or ultima_azione is None):
#                 # percentuale_bitcoin_usata = bitcoin_da_vendere / bitcoin
#                 euro_ricavati = bitcoin_da_vendere * prezzo_attuale
#                 bitcoin -= bitcoin_da_vendere
#                 liquidità += euro_ricavati
#                 azione = 'Vendita'
#                 ultima_azione = azione
#                 # percentuale_bitcoin_usata_rispetto_rif = percentuale_vendita
#                 # euro_spesi_o_ricavati = euro_ricavati
#                 # bitcoin_acquistati_o_venduti = bitcoin_da_vendere

#                 # Reset degli acquisti consecutivi dopo una vendita
#                 acquisti_consecutivi = []
#                 quantità_euro_totale = 0
#                 ultimo_prezzo_vendita = prezzo_attuale


#         # Aggiorna il valore totale del portafoglio
#         valore_totale_portafoglio = liquidità + (bitcoin * prezzo_attuale)

#         # variazione_perc_rispetto_riferimento_discesa = (prezzo_attuale / prezzo_riferimento_discesa - 1) * 100 if prezzo_riferimento_discesa else None
#         # variazione_perc_rispetto_riferimento_salita = (prezzo_attuale / prezzo_riferimento_salita - 1) * 100 if prezzo_riferimento_salita else None

#         # Salva la transazione nella lista
#         nuova_transazione = {
#             # 'data': data_odierna,
#             'prezzo': prezzo_attuale,
#             # 'variazione_giornaliera_prezzo_percentuale': round(variazione_giornaliera_prezzo_percentuale, 2),
#             # 'timestamp_riferimento_discesa': timestamp_riferimento_discesa,
#             # 'prezzo_riferimento_discesa': prezzo_riferimento_discesa,
#             # 'variazione_rispetto_riferimento_discesa': round((prezzo_attuale / prezzo_riferimento_discesa - 1) * 100, 2) if prezzo_riferimento_discesa else None,
#             # 'timestamp_riferimento_salita': timestamp_riferimento_salita,
#             # 'prezzo_riferimento_salita': prezzo_riferimento_salita,
#             # 'variazione_rispetto_riferimento_salita': round((prezzo_attuale / prezzo_riferimento_salita - 1) * 100, 2) if prezzo_riferimento_salita else None,
#             # 'azione': azione,
#             # 'percentuale_liquidità_usata': percentuale_liquidità_usata,
#             # 'percentuale_bitcoin_usata': percentuale_bitcoin_usata,
#             # 'quantità_euro': euro_spesi_o_ricavati,
#             # 'bitcoin_acquistati_o_venduti': bitcoin_acquistati_o_venduti,
#             # 'liquidità': liquidità,
#             'bitcoin': bitcoin,
#             'valore_totale_portafoglio': valore_totale_portafoglio
#         }
#         transazioni.append(nuova_transazione)

#     # Crea il DataFrame delle transazioni alla fine
#     df_transazioni = pd.DataFrame(transazioni)

#     return df_transazioni

### Algoritmo genetico

In [17]:
# def format_params(params):
#     return ", ".join(f"{key} = {value}" for key, value in params.items())

In [18]:
# # Definizione dello spazio dei parametri di ricerca
# # IMPORTANTE: INDICARE LE VARIABILI REALI CON LA VIRGOLA E QUELLE INTERE SENZA VIRGOLA
# space = {
#     'inverti_riferimento_acquisto': [True],
#     'inverti_riferimento_vendita': [True],
#     'acquisto_rispetto_liquidità_iniziale': [True],
#     'vendita_rispetto_massimo_btc_posseduti': [True],
#     'compra_pure': [True],
#     'vendi_pure': [True],
#     'soglia_vendita': (-4.0, 4.0),  # Real
#     'A_a': (0.1, 15),  # Real
#     'B_a': (0.001, 5),  # Real
#     'A_v': (0.1, 15),  # Real
#     'B_v': (0.001, 5),  # Real
#     'coefficienti_acquisto': ['custom_exp'],
#     'coefficienti_vendita': ['custom_exp'],
#     'compra_solo_in_discesa': [True],
#     'vendi_solo_in_salita': [False],
#     'soglia_acquisto_rispetto_riferimento': (-1.0, 1.0),  # Real # Forse meglio partire anche da -1.0
#     'soglia_vendita_rispetto_riferimento': (-1.0, 1.0),  # Real # Forse meglio partire anche da -1.0
#     'soglia_calcolo_variazioni_discesa': (0.0, 0.5),  # Real
#     'soglia_calcolo_variazioni_salita': (0.0, 0.5),  # Real
#     'perc_relativa': [True, False],
#     'finestra_minimo': (2, len(dati_da_usare[0])), # Int
#     'finestra_massimo': (2, len(dati_da_usare[0])), # Int
#     'nuovi_min_all_in': [True, False],
#     'nuovi_max_all_in': [True, False],
#     'soglia_acquisti_vicini': (0.0, 0.15), # Real
#     'soglia_vendite_vicine': (0.0, 0.15) # Real
# }

# # Imposta parametri di configurazione dell'algoritmo genetico
# population_size = 10          # Numero di individui nella popolazione iniziale
# elite_size = 3                # Numero di individui élite da mantenere
# crossover_probability = 1    # Probabilità di crossover
# mutation_probability = 0.5     # Probabilità di mutazione
# tournament_size = 4            # Numero di individui partecipanti a ogni torneo di selezione
# crossover_type = 'two-point'     # Tipo di crossover: 'single-point', 'two-point', 'uniform'
# indpb = [0.5] * len(space)             # Lista di probabilità di mutazione per ogni parametro
# selection_method = "custom"    # Selettore: "tournament", "roulette_wheel", "custom"
# elitism_method = "custom"      # Metodo di elitismo: "reduce", "custom" o altro, dove "custom" si assicura che ogni elite sostituisca uno dei peggiori solo se è migliore di lui (se metto altro succede come "custom", ma non si assicura di questo)
# optimize_direction = "maximize"  # Direzione di ottimizzazione: "minimize" o "maximize"
# offspring_ratio = 1.0          # Rapporto di sostituzione dei figli rispetto alla popolazione (di default 1.0); dovrebbe essere la percentuale di population_size da usare per creare i genitori che poi verranno accoppiati casualmente
# patience = 5                # Numero di generazioni senza miglioramenti prima di fermarsi
# num_generations = 50           # Numero di generazioni totali, se `patience=None`
# initial_individuals = [{'inverti_riferimento_acquisto': True, 'inverti_riferimento_vendita': True, 'acquisto_rispetto_liquidità_iniziale': True, 'vendita_rispetto_massimo_btc_posseduti': True, 'compra_pure': True, 'vendi_pure': True, 'soglia_vendita': -4, 'A_a': 0.22, 'B_a': 0.01, 'A_v': 0.26, 'B_v': 0.009, 'coefficienti_acquisto': 'custom_exp', 'coefficienti_vendita': 'custom_exp', 'compra_solo_in_discesa': True, 'vendi_solo_in_salita': False, 'soglia_acquisto_rispetto_riferimento': 0, 'soglia_vendita_rispetto_riferimento': -0.5, 'soglia_calcolo_variazioni_discesa': 0.08, 'soglia_calcolo_variazioni_salita': 0 , 'perc_relativa': True, 'finestra_minimo': 2, 'finestra_massimo': 2, 'nuovi_min_all_in': True, 'nuovi_max_all_in': False, 'soglia_acquisti_vicini': 0, 'soglia_vendite_vicine': 0}] * 9

# random_seed = 42               # Seme per la riproducibilità dei risultati
# use_stats = True               # Raccogliere le statistiche durante l'evoluzione
# verbosity = True               # Se True, mostrare output di ogni generazione

# random.seed(random_seed)
# np.random.seed(random_seed)

# # Definisci i parametri fissi
# parametri_fissi = {
#     'liquidità_iniziale': 1000,
#     'bitcoin_iniziali': 0
# }

# # Variabili globali per tenere traccia delle migliori e peggiori coppie
# best_pair = {'score': -np.inf, 'params': None, 'dataset': None}
# worst_pair = {'score': np.inf, 'params': None, 'dataset': None}

# def reinserimento_condizionato(population, elite_individuals, num_elite):
#     # Ordina la popolazione per fitness (dal migliore al peggiore)
#     population.sort(key=lambda x: x.fitness.values[0], reverse=(optimize_direction == "maximize"))

#     # Ottieni i peggiori individui della popolazione (da sostituire)
#     worst_individuals = population[-num_elite:]

#     # Ciclo per confrontare gli individui dell'élite con i peggiori
#     for i in range(num_elite):
#         # Se l'élite ha una fitness migliore del peggior individuo, lo sostituisce
#         if elite_individuals[i].fitness.values[0] > worst_individuals[i].fitness.values[0]:
#             print(f"Sostituzione: L'individuo peggiore con fitness {worst_individuals[i].fitness.values[0]} "
#                   f"viene sostituito dall'élite con fitness {elite_individuals[i].fitness.values[0]}.")
#             population[-(i + 1)] = elite_individuals[i]  # Sostituzione del peggior individuo con l'élite
#         else:
#             print(f"Nessuna sostituzione: L'individuo peggiore con fitness {worst_individuals[i].fitness.values[0]} "
#                   f"viene mantenuto perché la sua fitness è superiore o uguale a quella dell'élite.")

# # Funzione che crea un individuo in base allo space definito
# def create_individual():
#     individual = []
#     for param, values in space.items():
#         if isinstance(values, list):
#             # Variabile categorica o booleana
#             individual.append(random.choice(values))
#         elif isinstance(values[0], int):
#             # Variabile intera
#             individual.append(random.randint(*values))
#         else:
#             # Variabile reale
#             individual.append(random.uniform(*values))
#     return individual

# # Funzione di valutazione (funzione obiettivo)
# def eval_individual(individual, gen, individual_index):
#     global best_pair, worst_pair

#     # Ricava i parametri dell'individuo
#     params = {key: individual[i] for i, key in enumerate(space.keys())}
#     scores = []
#     for i, df in enumerate(dati_da_usare):
#         transactions = trading_perc_senza_micro_variazioni_velocizzato(df, **{**params, **parametri_fissi})
#         portfolio_values = transactions['valore_totale_portafoglio']
#         total_return_factor = portfolio_values.iloc[-1] / portfolio_values.iloc[0]
#         ideal_return = calcola_rendimento_portafoglio_ideale(df['Price'], *[parametri_fissi[key] for key in parametri_fissi.keys()])[0]
#         score = total_return_factor / ideal_return
#         scores.append(score)

#     avg_score = np.mean(scores)

#     # Stampa il numero progressivo dell'individuo e il punteggio
#     print(f"Generation {gen}:")
#     print(f"Individual {individual_index + 1}/{population_size}: {format_params(dict(zip(space.keys(), individual)))}")
#     print(f"Score: {avg_score}\n")

#     # Aggiorna best_pair e worst_pair
#     if avg_score > best_pair['score']:
#         best_pair = {'score': avg_score, 'params': params.copy(), 'dataset': i}
#     if avg_score < worst_pair['score']:
#         worst_pair = {'score': avg_score, 'params': params.copy(), 'dataset': i}

#     return (avg_score,)  # Restituisci la media dei punteggi

# # Funzione di selezione personalizzata
# def custom_selection(population, k, tournsize, optimize_direction):
#     """Selezione personalizzata che combina la selezione a torneo con la roulette wheel."""
#     chosen = []
#     for _ in range(k):
#         # Selezioniamo un sottoinsieme di `tournsize` individui, pesati per la loro fitness
#         selected = tools.selRoulette(population, tournsize)  # Roulette per selezionare `tournsize` individui
#         # Selezioniamo il migliore o il peggiore a seconda della direzione di ottimizzazione
#         if optimize_direction == "maximize":
#             winner = max(selected, key=lambda ind: ind.fitness.values[0])  # Miglior individuo vince il torneo
#         else:  # Se minimizziamo
#             winner = min(selected, key=lambda ind: ind.fitness.values[0])  # Peggior individuo (in minimizzazione)
#         chosen.append(winner)
#         # Log: debug selezione
#         print(f"Selezione: {winner} selezionato tra {selected}")
#     return chosen

# # Definizione del tipo di fitness e dell'individuo
# if optimize_direction == "maximize":
#     creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Massimizzazione
# else:
#     creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimizzazione

# creator.create("Individual", list, fitness=creator.FitnessMax if optimize_direction == "maximize" else creator.FitnessMin)

# # Funzione per la mutazione personalizzata
# def mutate_individual(individual, indpb):
#     """Applica la mutazione in base al tipo di parametro e a `indpb` per ogni gene."""
#     for i, (key, values) in enumerate(space.items()):
#         if random.random() < indpb[i]:  # Usa `indpb` specifica per quel parametro
#             if isinstance(values, list):  # Variabile categorica o booleana
#                 individual[i] = random.choice(values)
#             elif isinstance(values[0], int):  # Variabile intera
#                 individual[i] = random.randint(*values)
#             else:  # Variabile reale
#                 individual[i] = random.uniform(*values)
#             # Log: debug mutazione
#             print(f"Mutazione applicata al gene {key}: Nuovo valore = {individual[i]}")
#     return individual,

# # Crea la toolbox e registra le funzioni per l'algoritmo genetico
# toolbox = base.Toolbox()
# toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
# toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# toolbox.register("evaluate", eval_individual)
# toolbox.register("mutate", mutate_individual, indpb=indpb)

# def custom_cx_uniform(ind1, ind2, indpb):
#     """Esegue un crossover uniforme che può gestire una lista di probabilità."""
#     size = min(len(ind1), len(ind2))
#     for i in range(size):
#         if random.random() < indpb[i]:
#             ind1[i], ind2[i] = ind2[i], ind1[i]
#             # Log: debug crossover
#             print(f"Crossover tra i geni {i} ({list(space.keys())[i]}): ind1 = {ind1[i]}, ind2 = {ind2[i]}")
#     return ind1, ind2

# # Selezione: Usa il metodo selezionato dall'utente
# if selection_method == "roulette_wheel":
#     toolbox.register("select", tools.selRoulette)
# elif selection_method == "tournament":
#     toolbox.register("select", tools.selTournament, tournsize=tournament_size)
# elif selection_method == "custom":
#     toolbox.register("select", custom_selection, tournsize=tournament_size, optimize_direction=optimize_direction)
# else:
#     raise ValueError("selection_method deve essere 'tournament', 'roulette_wheel' o 'custom'.")

# # Impostazione del crossover in base al tipo scelto
# if crossover_type == 'single-point':
#     toolbox.register("mate", tools.cxOnePoint)
# elif crossover_type == 'two-point':
#     toolbox.register("mate", tools.cxTwoPoint)
# elif crossover_type == 'uniform':
#     toolbox.register("mate", custom_cx_uniform, indpb=indpb)  # Crossover uniforme con probabilità di scambio 'indpb'

# # Se vuoi raccogliere le statistiche
# if use_stats:
#     stats = tools.Statistics(lambda ind: ind.fitness.values)
#     stats.register("avg", np.mean)
#     stats.register("std", np.std)
#     stats.register("min", np.min)
#     stats.register("max", np.max)
# else:
#     stats = None

# # Funzione per applicare la logica di patience
# def patience_callback(population, hof, patience, best_fitness, patience_counter, previous_best_fitness):
#     current_best = tools.selBest(population, k=1)[0].fitness.values[0]
#     if optimize_direction == "maximize":
#         improvement_condition = current_best > best_fitness
#     else:
#         improvement_condition = current_best < best_fitness

#     if improvement_condition:
#         best_fitness = current_best
#         patience_counter = 0
#         print(f'Previous best fitness: {previous_best_fitness}')
#         print(f"New best fitness: {best_fitness}")
#         previous_best_fitness = best_fitness
#     else:
#         patience_counter += 1
#         print(f"No improvement, patience counter: {patience_counter}")
#     return best_fitness, patience_counter

# # Funzione principale per eseguire l'algoritmo genetico
# def run_genetic_algorithm(population_size, ngen, patience, initial_individuals=[]):
#     # Aggiungi gli individui specificati in initial_individuals alla popolazione
#     initial_population = [creator.Individual(ind) for ind in initial_individuals]
#     remaining_size = population_size - len(initial_individuals)
#     if remaining_size > 0:
#         initial_population.extend(toolbox.population(n=remaining_size))
#     else:
#         # Se len(initial_individuals) >= population_size, seleziona solo i migliori individui
#         initial_population.sort(key=lambda x: x.fitness.values[0], reverse=(optimize_direction == "maximize"))
#         initial_population = initial_population[:population_size]

#     population = initial_population
#     hof = tools.HallOfFame(1)  # Hall of Fame con 1 miglior individuo
#     best_fitness = -np.inf if optimize_direction == "maximize" else np.inf
#     previous_best_fitness = best_fitness
#     patience_counter = 0

#     # Valutazione degli individui iniziali
#     invalid_ind = [ind for ind in population if not ind.fitness.valid]
#     fitnesses = [toolbox.evaluate(ind, gen=0, individual_index=i) for i, ind in enumerate(invalid_ind)]  # Generazione 0
#     for ind, fit in zip(invalid_ind, fitnesses):
#         ind.fitness.values = fit

#     for gen in range(1, ngen + 1):  # Ciclo sulle generazioni successive
#         # Seleziona i genitori
#         offspring = toolbox.select(population, int(len(population) * offspring_ratio))

#         # Log: numero di individui selezionati prima dell'elitismo
#         print(f"Generazione {gen+1}: Numero individui (prima dell'elitismo): {len(offspring)}")

#         # Applicare crossover e mutazione agli individui selezionati
#         offspring = list(map(toolbox.clone, offspring))
#         for child1, child2 in zip(offspring[::2], offspring[1::2]):
#             if random.random() < crossover_probability:
#                 toolbox.mate(child1, child2)
#                 del child1.fitness.values
#                 del child2.fitness.values
#                 # Log: debug crossover
#                 print(f"Generazione {gen}: Crossover eseguito tra child1 e child2.")

#         n_mutants = 0
#         for mutant in offspring:
#             if random.random() < mutation_probability:
#                 n_mutants += 1
#                 toolbox.mutate(mutant)
#                 del mutant.fitness.values
#                 # Log: debug mutazione
#                 print(f"Generazione {gen}: Mutazione applicata a un individuo.")
#         print(f'Numero di mutazioni avvenute: {n_mutants}/{len(offspring)}')

#         # Valutazione degli individui alterati
#         invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
#         fitnesses = [toolbox.evaluate(ind, gen=gen, individual_index=i) for i, ind in enumerate(invalid_ind)]
#         for ind, fit in zip(invalid_ind, fitnesses):
#             ind.fitness.values = fit

#         # Log: numero di individui dopo la mutazione e il crossover
#         print(f"Generazione {gen}: Numero di figli (dopo crossover e mutazione): {len(offspring)}")

#         # Aggiorna popolazione con élite in base all'elitism_method
#         if elitism_method == "reduce":
#             # Metodo elitismo "reduce": aggiungiamo semplicemente gli élite alla popolazione senza sostituire nessuno
#             elite_individuals = tools.selBest(population, elite_size)
#             print(f"Generazione {gen}: Reinserimento di questi élite:\n- {elite_individuals}")
#             offspring.extend(map(toolbox.clone, elite_individuals))

#         elif elitism_method == "custom":
#             # Metodo elitismo "custom": usa il reinserimento condizionato per sostituire solo i peggiori se gli élite sono migliori
#             elite_individuals = tools.selBest(population, elite_size)
#             reinserimento_condizionato(offspring, elite_individuals, elite_size)

#         else:
#             # Metodo standard: sostituiamo i peggiori individui con gli élite
#             reverse_sort = (optimize_direction == "maximize")
#             offspring.sort(key=lambda ind: ind.fitness.values[0], reverse=reverse_sort)
#             offspring[-elite_size:] = map(toolbox.clone, tools.selBest(population, elite_size))

#         # Log: numero di individui finali dopo l'elitismo
#         print(f"Generazione {gen}: Numero finale di individui nella popolazione (dopo elitismo): {len(offspring)}")

#         population[:] = offspring

#         # Aggiorna Hall of Fame
#         hof.update(population)

#         # Statistiche opzionali
#         if stats:
#             record = stats.compile(population)
#             print(f"Stats for generation {gen}: {record}")

#         # Verbosità opzionale
#         if verbosity:
#             best_individual = tools.selBest(population, k=1)[0]
#             print(f"Generation {gen}: Best fitness {best_individual.fitness.values[0]}")
#             print(f"Best individual: {format_params(dict(zip(space.keys(), best_individual)))}")

#         # Controlla se il patience è impostato
#         if patience is not None:
#             best_fitness, patience_counter = patience_callback(population, hof, patience, best_fitness, patience_counter, previous_best_fitness)
#             if patience_counter >= patience:
#                 print("Patience limit reached, stopping evolution.")
#                 break

#     return population, hof  # Restituisci la popolazione finale e Hall of Fame

In [19]:
# # Esegui l'algoritmo genetico
# population, hof = run_genetic_algorithm(population_size, num_generations, patience, [j.values() for j in initial_individuals])

In [20]:
# # Alla fine dell'algoritmo genetico
# print("\nOptimization completed.")

# # Ottieni il miglior individuo dal Hall of Fame
# best_individual = hof[0]  # Miglior individuo
# best_fitness = best_individual.fitness.values[0]  # Fitness del miglior individuo

# # Stampa il miglior punteggio e i parametri corrispondenti
# print(f"Best average score: {best_fitness}, with parameters {format_params(best_pair['params'])}")

# # Stampa i dettagli del best_pair
# print("\nBest pair details:")
# print(f"Dataset: {best_pair['dataset']}")
# print(f"Score: {best_pair['score']}")
# print("Parameters:")
# print(format_params(best_pair['params']))

# # Stampa i dettagli del worst_pair
# print("\nWorst pair details:")
# print(f"Dataset: {worst_pair['dataset']}")
# print(f"Score: {worst_pair['score']}")
# print("Parameters:")
# print(format_params(worst_pair['params']))

# # Esegui la strategia con i migliori e peggiori parametri trovati
# print(f"\nBest pair transactions:\nPunteggio: {best_pair['score']}\nDataset: {best_pair['dataset']}\nParametri: {best_pair['params']}\nTransazioni:")
# best_transactions = trading_perc_senza_micro_variazioni_velocizzato(
#     dati_da_usare[best_pair['dataset']],
#     **{**best_pair['params'], **parametri_fissi}
# )
# display(best_transactions)

# print(f"\nWorst pair transactions:\nPunteggio: {worst_pair['score']}\nDataset: {worst_pair['dataset']}\nParametri: {worst_pair['params']}\nTransazioni:")
# worst_transactions = trading_perc_senza_micro_variazioni_velocizzato(
#     dati_da_usare[worst_pair['dataset']],
#     **{**worst_pair['params'], **parametri_fissi}
# )
# display(worst_transactions)

## Trading a potenze con matrice

In [21]:
# def normalizza_matrice(matrice, exception_cells):
#     """
#     Normalizza una matrice tenendo conto delle celle da escludere.
#     """
#     # Creiamo una copia della matrice originale per evitare modifiche in-place
#     matrice = matrice.copy()
    
#     # Imposta a zero le celle in exception_cells
#     for cella in exception_cells:
#         matrice[cella[0], cella[1]] = 0

#     # Normalizza la matrice in modo che la somma delle celle sia 1
#     somma_totale = np.sum(matrice)
    
#     if somma_totale == 0:
#         raise ValueError("La somma delle celle nella matrice, escluse le eccezioni, è 0. Non è possibile normalizzare.")
    
#     matrice /= somma_totale
#     return matrice

In [22]:
# def trading_potenze_con_matrice(dati, righe, colonne, matrice_liquidità, matrice_bitcoin, exception_cells, giorni_ridistribuzione, base_acquisto, base_vendita, soglia_vendita, max_exp_acquisto, max_exp_vendita, riferimento_iniziale_acquisto, compra_pure, liquidità_iniziale, bitcoin_iniziali):
#     if not matrice_liquidità[0]:
#         matrice_liquidità = np.full((righe, colonne), 1 / (righe * colonne))

#     if not matrice_bitcoin[0]:
#         matrice_bitcoin = np.full((righe, colonne), 1 / (righe * colonne))

#     # Controllo sugli max_exp_acquisto e max_exp_vendita
#     if not (isinstance(max_exp_acquisto, int) and max_exp_acquisto <= 0):
#         raise ValueError("max_exp_acquisto deve essere un intero non positivo.")
#     if not (isinstance(max_exp_vendita, int) and max_exp_vendita <= 0):
#         raise ValueError("max_exp_vendita deve essere un intero non positivo.")
    
#     # Convertiamo i dati di input in numpy array per efficienza
#     matrice_liquidità = np.array(matrice_liquidità)
#     matrice_bitcoin = np.array(matrice_bitcoin)
    
#     # Controlli iniziali: Verifichiamo che la somma delle celle faccia 1
#     if not np.isclose(np.sum(matrice_liquidità), 1) or not np.isclose(np.sum(matrice_bitcoin), 1):
#         raise ValueError("Le matrici non sono normalizzate correttamente, la somma delle loro celle deve essere 1.")
    
#     if base_acquisto < 1 or base_vendita < 1:
#         raise ValueError("Attenzione! base_vendita e base_acquisto devono essere entrambi almeno 1.")
    
#     # Normalizzazione delle matrici se ci sono eccezioni
#     if exception_cells:
#         matrice_liquidità = normalizza_matrice(matrice_liquidità, exception_cells)
#         matrice_bitcoin = normalizza_matrice(matrice_bitcoin, exception_cells)
    
#     # Estrazione dei dati sui prezzi
#     prezzi = dati['Price'].values
#     date = dati['Timestamp'].values
    
#     # Inizializzo le quantità parziali di liquidità e bitcoin per ogni strategia
#     liquidità_parziale = matrice_liquidità * liquidità_iniziale
#     bitcoin_parziale = matrice_bitcoin * bitcoin_iniziali
    
#     # Mantengo una copia della liquidità dopo l'ultima ridistribuzione, se necessario
#     liquidità_iniziale_ridistribuzione = np.copy(liquidità_parziale) if riferimento_iniziale_acquisto else None
    
#     # DataFrame per tenere traccia delle transazioni giornaliere
#     transazioni_giornaliere = []
    
#     # Variabili per tenere traccia di stato acquisto/vendita per ogni strategia
#     acquisto_contatori = np.zeros_like(matrice_liquidità)
#     vendita_contatori = np.zeros_like(matrice_bitcoin)
#     prezzo_medio_acquisto = np.zeros_like(matrice_bitcoin)
#     euro_spesi_acquisto = np.zeros_like(matrice_bitcoin)  # Tieni traccia degli euro spesi durante l'ultima scia di acquisti
#     in_scia_acquisto = np.zeros_like(matrice_bitcoin, dtype=bool)  # Flag per indicare se siamo in una scia di acquisti
    
#     # Loop sui giorni
#     for giorno in range(1, len(prezzi)):  # Inizia da 1 perché confrontiamo con il giorno precedente
#         prezzo_corrente = prezzi[giorno]
#         prezzo_precedente = prezzi[giorno - 1]
        
#         # Calcolo la liquidità e i bitcoin totali all'inizio del giorno
#         liquidità_inizio_giorno = np.sum(liquidità_parziale)
#         bitcoin_inizio_giorno = np.sum(bitcoin_parziale)
        
#         # Variabili per tenere traccia delle transazioni giornaliere
#         euro_spesi_giorno = 0
#         euro_guadagnati_giorno = 0
#         bitcoin_guadagnati_giorno = 0
#         bitcoin_venduti_giorno = 0
        
#         # Loop su tutte le strategie
#         for i in range(matrice_liquidità.shape[0]):
#             for j in range(matrice_liquidità.shape[1]):
#                 if (i, j) in exception_cells:
#                     continue
                
#                 # Acquisto: Se il prezzo scende e ho liquidità
#                 if prezzo_corrente < prezzo_precedente:
#                     # Determina la liquidità di riferimento
#                     if riferimento_iniziale_acquisto:
#                         liquidità_riferimento = liquidità_iniziale_ridistribuzione[i, j]
#                     else:
#                         liquidità_riferimento = liquidità_parziale[i, j]
                    
#                     # Determina la percentuale di acquisto in base alla progressione geometrica
#                     percentuale_acquisto = min(base_acquisto ** max_exp_acquisto, base_acquisto ** (max_exp_acquisto - i + acquisto_contatori[i, j]))
#                     euro_da_spendere = percentuale_acquisto * liquidità_riferimento
                    
#                     # Se la liquidità da spendere è maggiore della liquidità attuale
#                     if euro_da_spendere > liquidità_parziale[i, j]:
#                         if compra_pure:
#                             euro_acquisto = liquidità_parziale[i, j]  # Spendi tutto quello che hai
#                         else:
#                             continue  # Non comprare niente
#                     else:
#                         euro_acquisto = euro_da_spendere
                    
#                     # Aggiorna liquidità e bitcoin
#                     liquidità_parziale[i, j] -= euro_acquisto
#                     bitcoin_acquistati = euro_acquisto / prezzo_corrente
#                     bitcoin_parziale[i, j] += bitcoin_acquistati
                    
#                     # Aggiorna il prezzo medio pesato degli acquisti solo per la scia corrente
#                     euro_spesi_acquisto[i, j] += euro_acquisto
#                     prezzo_medio_acquisto[i, j] = (
#                         prezzo_medio_acquisto[i, j] * (euro_spesi_acquisto[i, j] - euro_acquisto) + euro_acquisto * prezzo_corrente
#                     ) / euro_spesi_acquisto[i, j]
                    
#                     # Aggiorna il contatore degli acquisti e delle transazioni giornaliere
#                     acquisto_contatori[i, j] += 1
#                     euro_spesi_giorno += euro_acquisto
#                     bitcoin_guadagnati_giorno += bitcoin_acquistati
#                     vendita_contatori[i, j] = 0  # Reset contatore vendite per nuova discesa
                
#                 # Vendita: Se il prezzo sale e ho bitcoin
#                 elif prezzo_corrente > prezzo_precedente and bitcoin_parziale[i, j] > 0:
#                     # Verifica se il prezzo corrente supera il prezzo medio pesato dell'ultima scia di acquisti
#                     if prezzo_corrente > prezzo_medio_acquisto[i, j] * (1 + soglia_vendita):
#                         # Reset della scia di acquisti e inizio della scia di vendite
#                         in_scia_acquisto[i, j] = False

#                         percentuale_vendita = min(base_vendita ** max_exp_vendita, base_vendita ** (max_exp_vendita - j + vendita_contatori[i, j]))
#                         bitcoin_da_vendere = percentuale_vendita * bitcoin_parziale[i, j]
                        
#                         # Aggiorna bitcoin e liquidità
#                         bitcoin_parziale[i, j] -= bitcoin_da_vendere
#                         euro_venduti = bitcoin_da_vendere * prezzo_corrente
#                         liquidità_parziale[i, j] += euro_venduti
                        
#                         # Aggiorna le transazioni giornaliere
#                         euro_guadagnati_giorno += euro_venduti
#                         bitcoin_venduti_giorno += bitcoin_da_vendere
                        
#                         # Incrementa il contatore delle vendite
#                         vendita_contatori[i, j] += 1
#                         acquisto_contatori[i, j] = 0  # Reset contatore acquisti per nuova salita

#         # Calcolo le percentuali di liquidità e bitcoin usati per le transazioni
#         percentuale_liquidità_usata = euro_spesi_giorno / liquidità_inizio_giorno if liquidità_inizio_giorno > 0 else 0
#         percentuale_bitcoin_venduti = bitcoin_venduti_giorno / bitcoin_inizio_giorno if bitcoin_inizio_giorno > 0 else 0
        
#         # Calcolo il valore del portafoglio totale alla fine del giorno
#         valore_liquidità_totale = np.sum(liquidità_parziale)
#         numero_totale_bitcoin = np.sum(bitcoin_parziale)
#         valore_bitcoin_totale = numero_totale_bitcoin * prezzo_corrente
#         valore_totale_giornaliero = valore_liquidità_totale + valore_bitcoin_totale
        
#         # Salvo i dati delle transazioni giornaliere
#         transazioni_giornaliere.append({
#             'data': date[giorno],
#             'prezzo': prezzo_corrente,
#             'euro spesi': euro_spesi_giorno,
#             'euro guadagnati': euro_guadagnati_giorno,
#             'bitcoin venduti': bitcoin_venduti_giorno,
#             'bitcoin guadagnati': bitcoin_guadagnati_giorno,
#             'percentuale liquidità usata': percentuale_liquidità_usata,
#             'percentuale bitcoin venduti': percentuale_bitcoin_venduti,
#             'liquidità totale': valore_liquidità_totale,
#             'bitcoin': numero_totale_bitcoin,
#             'valore bitcoin totali': valore_bitcoin_totale,
#             'valore_totale_portafoglio': valore_totale_giornaliero
#         })
        
#         # Ogni "giorni_ridistribuzione" giorni, facciamo la ridistribuzione
#         if (giorno + 1) % giorni_ridistribuzione == 0:
#             # Somma totale di liquidità e bitcoin attuali
#             liquidità_totale = np.sum(liquidità_parziale)
#             bitcoin_totale = np.sum(bitcoin_parziale)

#             # # Normalizzazione delle matrici se ci sono eccezioni
#             # if exception_cells:
#             #     matrice_liquidità = normalizza_matrice(matrice_liquidità, exception_cells)
#             #     matrice_bitcoin = normalizza_matrice(matrice_bitcoin, exception_cells)
            
#             # Ridistribuisco la liquidità e i bitcoin parziali in base alle matrici
#             liquidità_parziale = matrice_liquidità * liquidità_totale
#             bitcoin_parziale = matrice_bitcoin * bitcoin_totale
            
#             # Se riferimento_iniziale_acquisto è True, aggiorno la liquidità di riferimento
#             if riferimento_iniziale_acquisto:
#                 liquidità_iniziale_ridistribuzione = np.copy(liquidità_parziale)

#     # Converto le transazioni giornaliere in un DataFrame
#     df_transazioni = pd.DataFrame(transazioni_giornaliere)
    
#     return df_transazioni

### Algoritmo genetico

In [23]:
def format_params(params):
    return ", ".join(f"{key} = {value}" for key, value in params.items())

In [24]:
# # Definizione dello spazio dei parametri di ricerca
# # IMPORTANTE: INDICARE LE VARIABILI REALI CON LA VIRGOLA E QUELLE INTERE SENZA VIRGOLA (almeno l'estremo sinistro in entrambi i casi)
# space = {
#     'righe': (3, 15),
#     'colonne': (1, 15),
#     'matrice_liquidità': [[[]]], # qui come tipi di valori ho delle matrici, quindi va bene lasciarli dentro a una lista più esterna in modo che faccia un random.choice come se avessi un elenco di booleani o di stringhe
#     'matrice_bitcoin': [[[]]], # qui come tipi di valori ho delle matrici, quindi anche qui va bene lasciarli dentro a una lista più esterna
#     'exception_cells': [[], [(0, 0)], [(0, 0), (1, 0)]], # qui come tipi di valori ho delle liste (dove ogni lista contiene delle tuple), quindi anche qui va bene lasciarli dentro a una lista più esterna; la massima riga di eccezione può essere la minima riga che compare in "righe" meno 2 (contando così come sono le righe partendo da 1 e le eccezioni partendo da 0)
#     'giorni_ridistribuzione': (1, len(dati_da_usare[0])),
#     'base_acquisto': (1.0, 3.0),
#     'base_vendita': (1.0, 3.0),
#     'soglia_vendita': (-1.0, 1.0),
#     'max_exp_acquisto': (-3, 0),
#     'max_exp_vendita': (-3, 0),
#     'riferimento_iniziale_acquisto': [True, False],
#     'compra_pure': [True, False]
# }


# # Imposta parametri di configurazione dell'algoritmo genetico
# population_size = 300          # Numero di individui nella popolazione iniziale
# elite_size = 30                # Numero di individui élite da mantenere
# crossover_probability = 1    # Probabilità di crossover
# mutation_probability = 0.2     # Probabilità di mutazione
# tournament_size = 40            # Numero di individui partecipanti a ogni torneo di selezione
# crossover_type = 'uniform'     # Tipo di crossover: 'single-point', 'two-point', 'uniform'
# indpb = [0.5] * len(space)             # Lista di probabilità di mutazione per ogni parametro
# selection_method = "custom"    # Selettore: "tournament", "roulette_wheel", "custom"
# elitism_method = "custom"      # Metodo di elitismo: "reduce", "custom" o altro, dove "custom" si assicura che ogni elite sostituisca uno dei peggiori solo se è migliore di lui (se metto altro succede come "custom", ma non si assicura di questo)
# optimize_direction = "maximize"  # Direzione di ottimizzazione: "minimize" o "maximize"
# offspring_ratio = 1.0          # Rapporto di sostituzione dei figli rispetto alla popolazione (di default 1.0); dovrebbe essere la percentuale di population_size da usare per creare i genitori che poi verranno accoppiati casualmente
# patience = 5                # Numero di generazioni senza miglioramenti prima di fermarsi
# num_generations = 50           # Numero di generazioni totali, se `patience=None`
# initial_individuals = [{'righe': 3, 'colonne': 1, 'matrice_liquidità': [[]], 'matrice_bitcoin': [[]], 'exception_cells': [(0, 0), (1, 0)], 'giorni_ridistribuzione': 7, 'base_acquisto': 1.005, 'base_vendita': 1.08, 'soglia_vendita': -1, 'max_exp_acquisto': 0, 'max_exp_vendita': -1, 'riferimento_iniziale_acquisto': True, 'compra_pure': False}]# * (population_size-1)

# random_seed = 42               # Seme per la riproducibilità dei risultati
# use_stats = True               # Raccogliere le statistiche durante l'evoluzione
# verbosity = True               # Se True, mostrare output di ogni generazione

# random.seed(random_seed)
# np.random.seed(random_seed)

# # Definisci i parametri fissi
# parametri_fissi = {
#     'liquidità_iniziale': 1000,
#     'bitcoin_iniziali': 0
# }

# # Variabili globali per tenere traccia delle migliori e peggiori coppie
# best_pair = {'score': -np.inf, 'params': None, 'dataset': None}
# worst_pair = {'score': np.inf, 'params': None, 'dataset': None}

# def reinserimento_condizionato(population, elite_individuals, num_elite):
#     # Ordina la popolazione per fitness (dal migliore al peggiore)
#     population.sort(key=lambda x: x.fitness.values[0], reverse=(optimize_direction == "maximize"))

#     # Ottieni i peggiori individui della popolazione (da sostituire)
#     worst_individuals = population[-num_elite:]

#     # Ciclo per confrontare gli individui dell'élite con i peggiori
#     for i in range(num_elite):
#         # Se l'élite ha una fitness migliore del peggior individuo, lo sostituisce
#         if elite_individuals[i].fitness.values[0] > worst_individuals[i].fitness.values[0]:
#             print(f"Sostituzione: L'individuo peggiore con fitness {worst_individuals[i].fitness.values[0]} "
#                   f"viene sostituito dall'élite con fitness {elite_individuals[i].fitness.values[0]}.")
#             population[-(i + 1)] = elite_individuals[i]  # Sostituzione del peggior individuo con l'élite
#         else:
#             print(f"Nessuna sostituzione: L'individuo peggiore con fitness {worst_individuals[i].fitness.values[0]} "
#                   f"viene mantenuto perché la sua fitness è superiore o uguale a quella dell'élite.")

# # Funzione che crea un individuo in base allo space definito
# def create_individual():
#     individual = []
#     for param, values in space.items():
#         if isinstance(values, list):
#             # Variabile categorica o booleana
#             individual.append(random.choice(values))
#         elif isinstance(values[0], int):
#             # Variabile intera
#             individual.append(random.randint(*values))
#         else:
#             # Variabile reale
#             individual.append(random.uniform(*values))
#     return individual

# # Funzione di valutazione (funzione obiettivo)
# def eval_individual(individual, gen, individual_index):
#     global best_pair, worst_pair

#     # Ricava i parametri dell'individuo
#     params = {key: individual[i] for i, key in enumerate(space.keys())}
#     scores = []
#     for i, df in enumerate(dati_da_usare):
#         transactions = trading_potenze_con_matrice(df, **{**params, **parametri_fissi})
#         portfolio_values = transactions['valore_totale_portafoglio']
#         total_return_factor = portfolio_values.iloc[-1] / portfolio_values.iloc[0]
#         ideal_return = calcola_rendimento_portafoglio_ideale(df['Price'], *[parametri_fissi[key] for key in parametri_fissi.keys()])[0]
#         score = total_return_factor / ideal_return
#         scores.append(score)

#     avg_score = np.mean(scores)

#     # Stampa il numero progressivo dell'individuo e il punteggio
#     print(f"Generation {gen}:")
#     print(f"Individual {individual_index + 1}/{population_size}: {format_params(dict(zip(space.keys(), individual)))}")
#     print(f"Score: {avg_score}\n")

#     # Aggiorna best_pair e worst_pair
#     if avg_score > best_pair['score']:
#         best_pair = {'score': avg_score, 'params': params.copy(), 'dataset': i}
#     if avg_score < worst_pair['score']:
#         worst_pair = {'score': avg_score, 'params': params.copy(), 'dataset': i}

#     return (avg_score,)  # Restituisci la media dei punteggi

# # Funzione di selezione personalizzata
# def custom_selection(population, k, tournsize, optimize_direction):
#     """Selezione personalizzata che combina la selezione a torneo con la roulette wheel."""
#     chosen = []
#     for _ in range(k):
#         # Selezioniamo un sottoinsieme di `tournsize` individui, pesati per la loro fitness
#         selected = tools.selRoulette(population, tournsize)  # Roulette per selezionare `tournsize` individui
#         # Selezioniamo il migliore o il peggiore a seconda della direzione di ottimizzazione
#         if optimize_direction == "maximize":
#             winner = max(selected, key=lambda ind: ind.fitness.values[0])  # Miglior individuo vince il torneo
#         else:  # Se minimizziamo
#             winner = min(selected, key=lambda ind: ind.fitness.values[0])  # Peggior individuo (in minimizzazione)
#         chosen.append(winner)
#         # Log: debug selezione
#         print(f"Selezione: {winner} selezionato tra {selected}")
#     return chosen

# # Definizione del tipo di fitness e dell'individuo
# if optimize_direction == "maximize":
#     creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Massimizzazione
# else:
#     creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimizzazione

# creator.create("Individual", list, fitness=creator.FitnessMax if optimize_direction == "maximize" else creator.FitnessMin)

# # Funzione per la mutazione personalizzata
# def mutate_individual(individual, indpb):
#     """Applica la mutazione in base al tipo di parametro e a `indpb` per ogni gene."""
#     for i, (key, values) in enumerate(space.items()):
#         if random.random() < indpb[i]:  # Usa `indpb` specifica per quel parametro
#             if isinstance(values, list):  # Variabile categorica o booleana
#                 individual[i] = random.choice(values)
#             elif isinstance(values[0], int):  # Variabile intera
#                 individual[i] = random.randint(*values)
#             else:  # Variabile reale
#                 individual[i] = random.uniform(*values)
#             # Log: debug mutazione
#             print(f"Mutazione applicata al gene {key}: Nuovo valore = {individual[i]}")
#     return individual,

# # Crea la toolbox e registra le funzioni per l'algoritmo genetico
# toolbox = base.Toolbox()
# toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
# toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# toolbox.register("evaluate", eval_individual)
# toolbox.register("mutate", mutate_individual, indpb=indpb)

# def custom_cx_uniform(ind1, ind2, indpb):
#     """Esegue un crossover uniforme che può gestire una lista di probabilità."""
#     size = min(len(ind1), len(ind2))
#     for i in range(size):
#         if random.random() < indpb[i]:
#             ind1[i], ind2[i] = ind2[i], ind1[i]
#             # Log: debug crossover
#             print(f"Crossover tra i geni {i} ({list(space.keys())[i]}): ind1 = {ind1[i]}, ind2 = {ind2[i]}")
#     return ind1, ind2

# # Selezione: Usa il metodo selezionato dall'utente
# if selection_method == "roulette_wheel":
#     toolbox.register("select", tools.selRoulette)
# elif selection_method == "tournament":
#     toolbox.register("select", tools.selTournament, tournsize=tournament_size)
# elif selection_method == "custom":
#     toolbox.register("select", custom_selection, tournsize=tournament_size, optimize_direction=optimize_direction)
# else:
#     raise ValueError("selection_method deve essere 'tournament', 'roulette_wheel' o 'custom'.")

# # Impostazione del crossover in base al tipo scelto
# if crossover_type == 'single-point':
#     toolbox.register("mate", tools.cxOnePoint)
# elif crossover_type == 'two-point':
#     toolbox.register("mate", tools.cxTwoPoint)
# elif crossover_type == 'uniform':
#     toolbox.register("mate", custom_cx_uniform, indpb=indpb)  # Crossover uniforme con probabilità di scambio 'indpb'

# # Se vuoi raccogliere le statistiche
# if use_stats:
#     stats = tools.Statistics(lambda ind: ind.fitness.values)
#     stats.register("avg", np.mean)
#     stats.register("std", np.std)
#     stats.register("min", np.min)
#     stats.register("max", np.max)
# else:
#     stats = None

# # Funzione per applicare la logica di patience
# def patience_callback(population, best_fitness, patience_counter):
#     global previous_best_fitness
    
#     current_best = tools.selBest(population, k=1)[0].fitness.values[0]
#     if optimize_direction == "maximize":
#         improvement_condition = current_best > best_fitness
#     else:
#         improvement_condition = current_best < best_fitness

#     if improvement_condition:
#         best_fitness = current_best
#         patience_counter = 0
#         print(f'Previous best fitness: {previous_best_fitness}')
#         print(f"New best fitness: {best_fitness}")
#         previous_best_fitness = best_fitness
#     else:
#         patience_counter += 1
#         print(f"No improvement, patience counter: {patience_counter}")
#     return best_fitness, patience_counter

# # Funzione principale per eseguire l'algoritmo genetico
# def run_genetic_algorithm(population_size, ngen, patience, initial_individuals=[]):
#     global previous_best_fitness

#     # Aggiungi gli individui specificati in initial_individuals alla popolazione
#     initial_population = [creator.Individual(ind) for ind in initial_individuals]
#     remaining_size = population_size - len(initial_individuals)
#     if remaining_size > 0:
#         initial_population.extend(toolbox.population(n=remaining_size))
#     else:
#         # Se len(initial_individuals) >= population_size, seleziona solo i migliori individui
#         initial_population.sort(key=lambda x: x.fitness.values[0], reverse=(optimize_direction == "maximize"))
#         initial_population = initial_population[:population_size]

#     population = initial_population
#     hof = tools.HallOfFame(1)  # Hall of Fame con 1 miglior individuo
#     best_fitness = -np.inf if optimize_direction == "maximize" else np.inf
#     previous_best_fitness = best_fitness
#     patience_counter = 0

#     # Valutazione degli individui iniziali
#     invalid_ind = [ind for ind in population if not ind.fitness.valid]
#     fitnesses = [toolbox.evaluate(ind, gen=0, individual_index=i) for i, ind in enumerate(invalid_ind)]  # Generazione 0
#     for ind, fit in zip(invalid_ind, fitnesses):
#         ind.fitness.values = fit

#     for gen in range(1, ngen + 1):  # Ciclo sulle generazioni successive
#         # Seleziona i genitori
#         offspring = toolbox.select(population, int(len(population) * offspring_ratio))

#         # Log: numero di individui selezionati prima dell'elitismo
#         print(f"Generazione {gen+1}: Numero individui (prima dell'elitismo): {len(offspring)}")

#         # Applicare crossover e mutazione agli individui selezionati
#         offspring = list(map(toolbox.clone, offspring))
#         for child1, child2 in zip(offspring[::2], offspring[1::2]):
#             if random.random() < crossover_probability:
#                 toolbox.mate(child1, child2)
#                 del child1.fitness.values
#                 del child2.fitness.values
#                 # Log: debug crossover
#                 print(f"Generazione {gen}: Crossover eseguito tra child1 e child2.")

#         n_mutants = 0
#         for mutant in offspring:
#             if random.random() < mutation_probability:
#                 n_mutants += 1
#                 toolbox.mutate(mutant)
#                 del mutant.fitness.values
#                 # Log: debug mutazione
#                 print(f"Generazione {gen}: Mutazione applicata a un individuo.")
#         print(f'Numero di mutazioni avvenute: {n_mutants}/{len(offspring)}')

#         # Valutazione degli individui alterati
#         invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
#         fitnesses = [toolbox.evaluate(ind, gen=gen, individual_index=i) for i, ind in enumerate(invalid_ind)]
#         for ind, fit in zip(invalid_ind, fitnesses):
#             ind.fitness.values = fit

#         # Log: numero di individui dopo la mutazione e il crossover
#         print(f"Generazione {gen}: Numero di figli (dopo crossover e mutazione): {len(offspring)}")

#         # Aggiorna popolazione con élite in base all'elitism_method
#         if elitism_method == "reduce":
#             # Metodo elitismo "reduce": aggiungiamo semplicemente gli élite alla popolazione senza sostituire nessuno
#             elite_individuals = tools.selBest(population, elite_size)
#             print(f"Generazione {gen}: Reinserimento di questi élite:\n- {elite_individuals}")
#             offspring.extend(map(toolbox.clone, elite_individuals))

#         elif elitism_method == "custom":
#             # Metodo elitismo "custom": usa il reinserimento condizionato per sostituire solo i peggiori se gli élite sono migliori
#             elite_individuals = tools.selBest(population, elite_size)
#             reinserimento_condizionato(offspring, elite_individuals, elite_size)

#         else:
#             # Metodo standard: sostituiamo i peggiori individui con gli élite
#             reverse_sort = (optimize_direction == "maximize")
#             offspring.sort(key=lambda ind: ind.fitness.values[0], reverse=reverse_sort)
#             offspring[-elite_size:] = map(toolbox.clone, tools.selBest(population, elite_size))

#         # Log: numero di individui finali dopo l'elitismo
#         print(f"Generazione {gen}: Numero finale di individui nella popolazione (dopo elitismo): {len(offspring)}")

#         population[:] = offspring

#         # Aggiorna Hall of Fame
#         hof.update(population)

#         # Statistiche opzionali
#         if stats:
#             record = stats.compile(population)
#             print(f"Stats for generation {gen}: {record}")

#         # Verbosità opzionale
#         if verbosity:
#             best_individual = tools.selBest(population, k=1)[0]
#             print(f"Generation {gen}: Best fitness {best_individual.fitness.values[0]}")
#             print(f"Best individual: {format_params(dict(zip(space.keys(), best_individual)))}")

#         # Controlla se il patience è impostato
#         if patience is not None:
#             best_fitness, patience_counter = patience_callback(population, best_fitness, patience_counter)
#             if patience_counter >= patience:
#                 print("Patience limit reached, stopping evolution.")
#                 break

#     return population, hof  # Restituisci la popolazione finale e Hall of Fame

In [25]:
# # Esegui l'algoritmo genetico
# population, hof = run_genetic_algorithm(population_size, num_generations, patience, [j.values() for j in initial_individuals])
# # Best individual: righe = 7, colonne = 1, matrice_liquidità = [[]], matrice_bitcoin = [[]], exception_cells = [(0, 0), (1, 0)], giorni_ridistribuzione = 25, base_acquisto = 2.7310444617128247, base_vendita = 1.0398559797771052, soglia_vendita = 0.03951843698407398, max_exp_acquisto = -1, max_exp_vendita = 0, riferimento_iniziale_acquisto = True, compra_pure = True
# # Best fitness 0.5083532147634425

In [26]:
# # Alla fine dell'algoritmo genetico
# print("\nOptimization completed.")

# # Ottieni il miglior individuo dal Hall of Fame
# best_individual = hof[0]  # Miglior individuo
# best_fitness = best_individual.fitness.values[0]  # Fitness del miglior individuo

# # Stampa il miglior punteggio e i parametri corrispondenti
# print(f"Best average score: {best_fitness}, with parameters {format_params(best_pair['params'])}")

# # Stampa i dettagli del best_pair
# print("\nBest pair details:")
# print(f"Dataset: {best_pair['dataset']}")
# print(f"Score: {best_pair['score']}")
# print("Parameters:")
# print(format_params(best_pair['params']))

# # Stampa i dettagli del worst_pair
# print("\nWorst pair details:")
# print(f"Dataset: {worst_pair['dataset']}")
# print(f"Score: {worst_pair['score']}")
# print("Parameters:")
# print(format_params(worst_pair['params']))

# # Esegui la strategia con i migliori e peggiori parametri trovati
# print(f"\nBest pair transactions:\nPunteggio: {best_pair['score']}\nDataset: {best_pair['dataset']}\nParametri: {best_pair['params']}\nTransazioni:")
# best_transactions = trading_potenze_con_matrice(
#     dati_da_usare[best_pair['dataset']],
#     **{**best_pair['params'], **parametri_fissi}
# )
# display(best_transactions)

# print(f"\nWorst pair transactions:\nPunteggio: {worst_pair['score']}\nDataset: {worst_pair['dataset']}\nParametri: {worst_pair['params']}\nTransazioni:")
# worst_transactions = trading_potenze_con_matrice(
#     dati_da_usare[worst_pair['dataset']],
#     **{**worst_pair['params'], **parametri_fissi}
# )
# display(worst_transactions)

## Strategia vendita Bitcoin

In [27]:
def calcola_max_drawdown_relativo_prezzo(prezzi, prezzi_paracadute):
    """
    Calcola il massimo drawdown dei prezzi, cioè la massima discesa percentuale del prezzo
    rispetto al massimo precedente, ma relativamente al prezzo paracadute

    Parameters:
    - prezzi (pd.Series or list): Serie temporale dei prezzi.

    Returns:
    - max_drawdown (float): Il massimo drawdown percentuale sui prezzi.
    - drawdown_serie (pd.Series): La serie temporale del drawdown percentuale giorno per giorno.
    """
    # Trasformo i prezzi in una Serie pandas se non lo è già
    prezzi = pd.Series(prezzi)
    
    # Calcola i massimi cumulativi dei prezzi fino a ogni punto
    max_prezzi_cumulativi = prezzi.cummax()
    
    # Calcola il drawdown percentuale relativo
    drawdown_serie = (max_prezzi_cumulativi - prezzi) / (max_prezzi_cumulativi - prezzi_paracadute)
    
    # Trova il massimo drawdown
    max_drawdown = drawdown_serie.max()
    
    # Ritorna il max drawdown
    return max_drawdown

def mappatura_discesa_vendita(percentuale_discesa_relativa, A, B):
    return percentuale_discesa_relativa ** A / (B + (1 - B) * percentuale_discesa_relativa ** A)

In [28]:
def strategia_vendita(dati, soglia_sopportabile, vendita_rispetto_inizio, paracadute_variabile, sopportazione_variabile, pct_paracadute, A, B, liquidità_iniziale, bitcoin_iniziali, prezzo_medio_vendite_iniziale):

    prezzo_paracadute = prezzo_medio_vendite_iniziale * (1 + pct_paracadute)

    # Variabili iniziali
    btc_posseduti = bitcoin_iniziali
    liquidità = liquidità_iniziale
    massimo_prezzo_raggiunto = dati['Price'].iloc[0] # Per tenere traccia del massimo raggiunto
    massimo_precedente = dati['Price'].iloc[0]

    prezzi_paracadute = [prezzo_paracadute]

    # DataFrame per registrare le transazioni giornaliere
    transazioni = []

    # Itera sulle righe del DataFrame (supponendo ordinato per data)
    for i in range(1, len(dati)):
        # Aggiorna il massimo prezzo raggiunto
        prezzo_attuale = dati["Price"].iloc[i]

        if max(massimo_prezzo_raggiunto, prezzo_attuale) > massimo_prezzo_raggiunto:
            massimo_precedente = massimo_prezzo_raggiunto
            massimo_prezzo_raggiunto = max(massimo_prezzo_raggiunto, prezzo_attuale)
            if paracadute_variabile:
                prezzo_paracadute = prezzo_paracadute * massimo_prezzo_raggiunto / massimo_precedente #prezzo_attuale / rapporto_prezzo_e_paracadute

            prezzi_paracadute.append(prezzo_paracadute)

            if sopportazione_variabile:
                max_drawdown_relativo_prezzo = calcola_max_drawdown_relativo_prezzo(dati["Price"].iloc[:i+1], prezzi_paracadute)
                # Cambio la soglia sopportabile solo se il max drawdown relativo che c'è stato in passato è maggiore della soglia sopportabile attuale
                if max_drawdown_relativo_prezzo > soglia_sopportabile:
                    soglia_sopportabile = max_drawdown_relativo_prezzo
        else:
            prezzi_paracadute.append(prezzo_paracadute)

        azione = "-"
        btc_da_vendere = 0
        liquidità_guadagnata = 0
        pct_btc_venduta_attuali = '-'
        pct_btc_venduta_iniziali = '-'

        # Calcola la percentuale attuale di discesa rispetto alla base di riferimento
        percentuale_discesa_relativa = (massimo_prezzo_raggiunto - prezzo_attuale) / (massimo_prezzo_raggiunto - prezzo_paracadute)
        if percentuale_discesa_relativa > 1:
            percentuale_discesa_relativa = 1
        
        # Controlla se il prezzo attuale è in discesa rispetto al prezzo di ieri
        prezzo_ieri = dati["Price"].iloc[i - 1]
        prezzo_in_discesa = prezzo_attuale < prezzo_ieri
        variazione_prezzo = (prezzo_attuale - prezzo_ieri) / prezzo_ieri

        # Condizione di vendita
        if percentuale_discesa_relativa > soglia_sopportabile and prezzo_in_discesa:
            # Calcola la percentuale di BTC da vendere
            percentuale_da_vendere = mappatura_discesa_vendita(percentuale_discesa_relativa, A, B) if mappatura_discesa_vendita(percentuale_discesa_relativa, A, B) <= 1 else 1
            
            if vendita_rispetto_inizio:
                # Calcola la quantità di BTC da vendere rispetto ai BTC iniziali
                btc_da_vendere = bitcoin_iniziali * percentuale_da_vendere
            else:
                # Calcola la quantità di BTC da vendere rispetto ai BTC attualmente posseduti
                btc_da_vendere = btc_posseduti * percentuale_da_vendere

            # Verifica di non vendere più BTC di quanti ne possieda
            if btc_da_vendere > btc_posseduti:
                btc_da_vendere = btc_posseduti

            if btc_posseduti > 0:
                # Effettua la vendita
                liquidità_guadagnata = btc_da_vendere * prezzo_attuale
                liquidità += liquidità_guadagnata
                btc_posseduti_prima = btc_posseduti
                btc_posseduti -= btc_da_vendere
                azione = "Vendita"

                # Calcola le percentuali di BTC venduta
                pct_btc_venduta_attuali = (btc_da_vendere / btc_posseduti_prima)
                pct_btc_venduta_iniziali = (btc_da_vendere / bitcoin_iniziali)

        # Valore totale del portafoglio
        valore_totale_portafoglio = liquidità + (btc_posseduti * prezzo_attuale)

        # Registra i dati nel dataframe delle transazioni
        transazioni.append({
            # "Timestamp": dati["Timestamp"].iloc[i],
            # "prezzo": prezzo_attuale,
            # "max_prezzo": massimo_prezzo_raggiunto,
            # "prezzo_paracadute": prezzo_paracadute,
            # "soglia_sopportabile": soglia_sopportabile,
            # "variazione % prezzo": variazione_prezzo,  # Convertito in percentuale
            # "% discesa relativa": percentuale_discesa_relativa if prezzo_in_discesa else "-",
            # "azione": azione,
            # "quantità BTC venduta": btc_da_vendere,
            # "% BTC venduta attuali": pct_btc_venduta_attuali,
            # "% BTC venduta iniziali": pct_btc_venduta_iniziali,
            # "liquidità guadagnata": liquidità_guadagnata,
            # "liquidità": liquidità,
            # "bitcoin": btc_posseduti,
            "valore_totale_portafoglio": valore_totale_portafoglio
        })

    # Crea un DataFrame dalle transazioni
    df_transazioni = pd.DataFrame(transazioni)

    # Mostra le prime righe del DataFrame delle transazioni
    return df_transazioni

### Algoritmo genetico

In [29]:
def format_params(params):
    return ", ".join(f"{key} = {value}" for key, value in params.items())

In [30]:
# Definizione dello spazio dei parametri di ricerca
# IMPORTANTE: INDICARE LE VARIABILI REALI CON LA VIRGOLA E QUELLE INTERE SENZA VIRGOLA (almeno l'estremo sinistro in entrambi i casi)
space = {
    'soglia_sopportabile': (0.0, 1.0),
    'vendita_rispetto_inizio': [True, False],
    'paracadute_variabile': [True, False],
    'sopportazione_variabile': [True, False],
    'pct_paracadute': (0.0, 1.0),
    'A': (0.0, 20.0),
    'B': (0.0, 100.0)
}

# Imposta parametri di configurazione dell'algoritmo genetico
population_size = 1000          # Numero di individui nella popolazione iniziale
elite_size = 30                # Numero di individui élite da mantenere
crossover_probability = 1    # Probabilità di crossover
mutation_probability = 0.2     # Probabilità di mutazione
tournament_size = 40            # Numero di individui partecipanti a ogni torneo di selezione
crossover_type = 'single-point'     # Tipo di crossover: 'single-point', 'two-point', 'uniform'
indpb = [0.5] * len(space)             # Lista di probabilità di mutazione per ogni parametro
selection_method = "custom"    # Selettore: "tournament", "roulette_wheel", "custom"
elitism_method = "custom"      # Metodo di elitismo: "reduce", "custom" o altro, dove "custom" si assicura che ogni elite sostituisca uno dei peggiori solo se è migliore di lui (se metto altro succede come "custom", ma non si assicura di questo)
optimize_direction = "maximize"  # Direzione di ottimizzazione: "minimize" o "maximize"
offspring_ratio = 1.0          # Rapporto di sostituzione dei figli rispetto alla popolazione (di default 1.0); dovrebbe essere la percentuale di population_size da usare per creare i genitori che poi verranno accoppiati casualmente
patience = 15                # Numero di generazioni senza miglioramenti prima di fermarsi
num_generations = 50           # Numero di generazioni totali, se patience=None
initial_individuals = [{'soglia_sopportabile':0.15, 'vendita_rispetto_inizio':True, 'paracadute_variabile':True, 'sopportazione_variabile':True, 'pct_paracadute':0.0, 'A':0.0, 'B':0.0}] * int((population_size/2))

random_seed = 42               # Seme per la riproducibilità dei risultati
use_stats = True               # Raccogliere le statistiche durante l'evoluzione
verbosity = True               # Se True, mostrare output di ogni generazione

random.seed(random_seed)
np.random.seed(random_seed)

# Definisci i parametri fissi
parametri_fissi = {
    'liquidità_iniziale': 0,
    'bitcoin_iniziali': 0.02478146,
    'prezzo_medio_vendite_iniziale': 20000
}

# Variabili globali per tenere traccia delle migliori e peggiori coppie
best_pair = {'score': -np.inf, 'params': None, 'dataset': None}
worst_pair = {'score': np.inf, 'params': None, 'dataset': None}

def reinserimento_condizionato(population, elite_individuals, num_elite):
    # Ordina la popolazione per fitness (dal migliore al peggiore)
    population.sort(key=lambda x: x.fitness.values[0], reverse=(optimize_direction == "maximize"))

    # Ottieni i peggiori individui della popolazione (da sostituire)
    worst_individuals = population[-num_elite:]

    # Ciclo per confrontare gli individui dell'élite con i peggiori
    for i in range(num_elite):
        # Se l'élite ha una fitness migliore del peggior individuo, lo sostituisce
        if elite_individuals[i].fitness.values[0] > worst_individuals[i].fitness.values[0]:
            print(f"Sostituzione: L'individuo peggiore con fitness {worst_individuals[i].fitness.values[0]} "
                  f"viene sostituito dall'élite con fitness {elite_individuals[i].fitness.values[0]}.")
            population[-(i + 1)] = elite_individuals[i]  # Sostituzione del peggior individuo con l'élite
        else:
            print(f"Nessuna sostituzione: L'individuo peggiore con fitness {worst_individuals[i].fitness.values[0]} "
                  f"viene mantenuto perché la sua fitness è superiore o uguale a quella dell'élite.")

# Funzione che crea un individuo in base allo space definito
def create_individual():
    individual = []
    for param, values in space.items():
        if isinstance(values, list):
            # Variabile categorica o booleana
            individual.append(random.choice(values))
        elif isinstance(values[0], int):
            # Variabile intera
            individual.append(random.randint(*values))
        else:
            # Variabile reale
            individual.append(random.uniform(*values))
    return individual

# Funzione di valutazione (funzione obiettivo)
def eval_individual(individual, gen, individual_index):
    global best_pair, worst_pair

    # Ricava i parametri dell'individuo
    params = {key: individual[i] for i, key in enumerate(space.keys())}
    scores = []
    for i, df in enumerate(dati_da_usare):
        transactions = strategia_vendita(df, **{**params, **parametri_fissi})
        portfolio_values = transactions['valore_totale_portafoglio']
        # total_return_factor = portfolio_values.iloc[-1] / portfolio_values.iloc[0]
        # ideal_return = calcola_rendimento_portafoglio_ideale(df['Price'], *[parametri_fissi[key] for key in list(parametri_fissi.keys())[:2]])[0]
        score = portfolio_values.iloc[-1] / (parametri_fissi['liquidità_iniziale'] + parametri_fissi['bitcoin_iniziali'] * dati['Price'].iloc[0]) # total_return_factor / ideal_return
        scores.append(score)

    avg_score = np.mean(scores)

    # Stampa il numero progressivo dell'individuo e il punteggio
    print(f"Generation {gen}:")
    print(f"Individual {individual_index + 1}/{population_size}: {format_params(dict(zip(space.keys(), individual)))}")
    print(f"Score: {avg_score}\n")

    # Aggiorna best_pair e worst_pair
    if avg_score > best_pair['score']:
        best_pair = {'score': avg_score, 'params': params.copy(), 'dataset': i}
    if avg_score < worst_pair['score']:
        worst_pair = {'score': avg_score, 'params': params.copy(), 'dataset': i}

    return (avg_score,)  # Restituisci la media dei punteggi

# Funzione di selezione personalizzata
def custom_selection(population, k, tournsize, optimize_direction):
    """Selezione personalizzata che combina la selezione a torneo con la roulette wheel."""
    chosen = []
    for _ in range(k):
        # Selezioniamo un sottoinsieme di `tournsize` individui, pesati per la loro fitness
        selected = tools.selRoulette(population, tournsize)  # Roulette per selezionare `tournsize` individui
        # Selezioniamo il migliore o il peggiore a seconda della direzione di ottimizzazione
        if optimize_direction == "maximize":
            winner = max(selected, key=lambda ind: ind.fitness.values[0])  # Miglior individuo vince il torneo
        else:  # Se minimizziamo
            winner = min(selected, key=lambda ind: ind.fitness.values[0])  # Peggior individuo (in minimizzazione)
        chosen.append(winner)
        # Log: debug selezione
        print(f"Selezione: {winner} selezionato tra {selected}")
    return chosen

# Definizione del tipo di fitness e dell'individuo
if optimize_direction == "maximize":
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))  # Massimizzazione
else:
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))  # Minimizzazione

creator.create("Individual", list, fitness=creator.FitnessMax if optimize_direction == "maximize" else creator.FitnessMin)

# Funzione per la mutazione personalizzata
def mutate_individual(individual, indpb):
    """Applica la mutazione in base al tipo di parametro e a `indpb` per ogni gene."""
    for i, (key, values) in enumerate(space.items()):
        if random.random() < indpb[i]:  # Usa `indpb` specifica per quel parametro
            if isinstance(values, list):  # Variabile categorica o booleana
                individual[i] = random.choice(values)
            elif isinstance(values[0], int):  # Variabile intera
                individual[i] = random.randint(*values)
            else:  # Variabile reale
                individual[i] = random.uniform(*values)
            # Log: debug mutazione
            print(f"Mutazione applicata al gene {key}: Nuovo valore = {individual[i]}")
    return individual,

# Crea la toolbox e registra le funzioni per l'algoritmo genetico
toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", eval_individual)
toolbox.register("mutate", mutate_individual, indpb=indpb)

def custom_cx_uniform(ind1, ind2, indpb):
    """Esegue un crossover uniforme che può gestire una lista di probabilità."""
    size = min(len(ind1), len(ind2))
    for i in range(size):
        if random.random() < indpb[i]:
            ind1[i], ind2[i] = ind2[i], ind1[i]
            # Log: debug crossover
            print(f"Crossover tra i geni {i} ({list(space.keys())[i]}): ind1 = {ind1[i]}, ind2 = {ind2[i]}")
    return ind1, ind2

# Selezione: Usa il metodo selezionato dall'utente
if selection_method == "roulette_wheel":
    toolbox.register("select", tools.selRoulette)
elif selection_method == "tournament":
    toolbox.register("select", tools.selTournament, tournsize=tournament_size)
elif selection_method == "custom":
    toolbox.register("select", custom_selection, tournsize=tournament_size, optimize_direction=optimize_direction)
else:
    raise ValueError("selection_method deve essere 'tournament', 'roulette_wheel' o 'custom'.")

# Impostazione del crossover in base al tipo scelto
if crossover_type == 'single-point':
    toolbox.register("mate", tools.cxOnePoint)
elif crossover_type == 'two-point':
    toolbox.register("mate", tools.cxTwoPoint)
elif crossover_type == 'uniform':
    toolbox.register("mate", custom_cx_uniform, indpb=indpb)  # Crossover uniforme con probabilità di scambio 'indpb'

# Se vuoi raccogliere le statistiche
if use_stats:
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
else:
    stats = None

# Funzione per applicare la logica di patience
def patience_callback(population, best_fitness, patience_counter):
    global previous_best_fitness
    
    current_best = tools.selBest(population, k=1)[0].fitness.values[0]
    if optimize_direction == "maximize":
        improvement_condition = current_best > best_fitness
    else:
        improvement_condition = current_best < best_fitness

    if improvement_condition:
        best_fitness = current_best
        patience_counter = 0
        print(f'Previous best fitness: {previous_best_fitness}')
        print(f"New best fitness: {best_fitness}")
        previous_best_fitness = best_fitness
    else:
        patience_counter += 1
        print(f"No improvement, patience counter: {patience_counter}")
    return best_fitness, patience_counter

# Funzione principale per eseguire l'algoritmo genetico
def run_genetic_algorithm(population_size, ngen, patience, initial_individuals=[]):
    global previous_best_fitness

    # Aggiungi gli individui specificati in initial_individuals alla popolazione
    initial_population = [creator.Individual(ind) for ind in initial_individuals]
    remaining_size = population_size - len(initial_individuals)
    if remaining_size > 0:
        initial_population.extend(toolbox.population(n=remaining_size))
    else:
        # Se len(initial_individuals) >= population_size, seleziona solo i migliori individui
        initial_population.sort(key=lambda x: x.fitness.values[0], reverse=(optimize_direction == "maximize"))
        initial_population = initial_population[:population_size]

    population = initial_population
    hof = tools.HallOfFame(1)  # Hall of Fame con 1 miglior individuo
    best_fitness = -np.inf if optimize_direction == "maximize" else np.inf
    previous_best_fitness = best_fitness
    patience_counter = 0

    # Valutazione degli individui iniziali
    invalid_ind = [ind for ind in population if not ind.fitness.valid]
    fitnesses = [toolbox.evaluate(ind, gen=0, individual_index=i) for i, ind in enumerate(invalid_ind)]  # Generazione 0
    for ind, fit in zip(invalid_ind, fitnesses):
        ind.fitness.values = fit

    for gen in range(1, ngen + 1):  # Ciclo sulle generazioni successive
        # Seleziona i genitori
        offspring = toolbox.select(population, int(len(population) * offspring_ratio))

        # Log: numero di individui selezionati prima dell'elitismo
        print(f"Generazione {gen+1}: Numero individui (prima dell'elitismo): {len(offspring)}")

        # Applicare crossover e mutazione agli individui selezionati
        offspring = list(map(toolbox.clone, offspring))
        for child1, child2 in zip(offspring[::2], offspring[1::2]):
            if random.random() < crossover_probability:
                toolbox.mate(child1, child2)
                del child1.fitness.values
                del child2.fitness.values
                # Log: debug crossover
                print(f"Generazione {gen}: Crossover eseguito tra child1 e child2.")

        n_mutants = 0
        for mutant in offspring:
            if random.random() < mutation_probability:
                n_mutants += 1
                toolbox.mutate(mutant)
                del mutant.fitness.values
                # Log: debug mutazione
                print(f"Generazione {gen}: Mutazione applicata a un individuo.")
        print(f'Numero di mutazioni avvenute: {n_mutants}/{len(offspring)}')

        # Valutazione degli individui alterati
        invalid_ind = [ind for ind in offspring if not ind.fitness.valid]
        fitnesses = [toolbox.evaluate(ind, gen=gen, individual_index=i) for i, ind in enumerate(invalid_ind)]
        for ind, fit in zip(invalid_ind, fitnesses):
            ind.fitness.values = fit

        # Log: numero di individui dopo la mutazione e il crossover
        print(f"Generazione {gen}: Numero di figli (dopo crossover e mutazione): {len(offspring)}")

        # Aggiorna popolazione con élite in base all'elitism_method
        if elitism_method == "reduce":
            # Metodo elitismo "reduce": aggiungiamo semplicemente gli élite alla popolazione senza sostituire nessuno
            elite_individuals = tools.selBest(population, elite_size)
            print(f"Generazione {gen}: Reinserimento di questi élite:\n- {elite_individuals}")
            offspring.extend(map(toolbox.clone, elite_individuals))

        elif elitism_method == "custom":
            # Metodo elitismo "custom": usa il reinserimento condizionato per sostituire solo i peggiori se gli élite sono migliori
            elite_individuals = tools.selBest(population, elite_size)
            reinserimento_condizionato(offspring, elite_individuals, elite_size)

        else:
            # Metodo standard: sostituiamo i peggiori individui con gli élite
            reverse_sort = (optimize_direction == "maximize")
            offspring.sort(key=lambda ind: ind.fitness.values[0], reverse=reverse_sort)
            offspring[-elite_size:] = map(toolbox.clone, tools.selBest(population, elite_size))

        # Log: numero di individui finali dopo l'elitismo
        print(f"Generazione {gen}: Numero finale di individui nella popolazione (dopo elitismo): {len(offspring)}")

        population[:] = offspring

        # Aggiorna Hall of Fame
        hof.update(population)

        # Statistiche opzionali
        if stats:
            record = stats.compile(population)
            print(f"Stats for generation {gen}: {record}")

        # Verbosità opzionale
        if verbosity:
            best_individual = tools.selBest(population, k=1)[0]
            print(f"Generation {gen}: Best fitness {best_individual.fitness.values[0]}")
            print(f"Best individual: {format_params(dict(zip(space.keys(), best_individual)))}")

        # Controlla se il patience è impostato
        if patience is not None:
            best_fitness, patience_counter = patience_callback(population, best_fitness, patience_counter)
            if patience_counter >= patience:
                print("Patience limit reached, stopping evolution.")
                break

    return population, hof  # Restituisci la popolazione finale e Hall of Fame

In [ ]:
# Esegui l'algoritmo genetico
population, hof = run_genetic_algorithm(population_size, num_generations, patience, [j.values() for j in initial_individuals])
# Best individual: righe = 7, colonne = 1, matrice_liquidità = [[]], matrice_bitcoin = [[]], exception_cells = [(0, 0), (1, 0)], giorni_ridistribuzione = 25, base_acquisto = 2.7310444617128247, base_vendita = 1.0398559797771052, soglia_vendita = 0.03951843698407398, max_exp_acquisto = -1, max_exp_vendita = 0, riferimento_iniziale_acquisto = True, compra_pure = True
# Best fitness 0.5083532147634425

In [ ]:
# Stats for generation 16: {'avg': np.float64(1.5666187968878893), 'std': np.float64(0.10786963397251519), 'min': np.float64(1.0198208166576217), 'max': np.float64(1.6035375630992346)}
# Generation 16: Best fitness 1.6035375630992346
# Best individual: soglia_sopportabile = 0.15, vendita_rispetto_inizio = True, paracadute_variabile = True, sopportazione_variabile = True, pct_paracadute = 0.0, A = 0.0, B = 0.0
# No improvement, patience counter: 15

In [32]:
# Alla fine dell'algoritmo genetico
print("\nOptimization completed.")

# Ottieni il miglior individuo dal Hall of Fame
best_individual = hof[0]  # Miglior individuo
best_fitness = best_individual.fitness.values[0]  # Fitness del miglior individuo

# Stampa il miglior punteggio e i parametri corrispondenti
print(f"Best average score: {best_fitness}, with parameters {format_params(best_pair['params'])}")

# Stampa i dettagli del best_pair
print("\nBest pair details:")
print(f"Dataset: {best_pair['dataset']}")
print(f"Score: {best_pair['score']}")
print("Parameters:")
print(format_params(best_pair['params']))

# Stampa i dettagli del worst_pair
print("\nWorst pair details:")
print(f"Dataset: {worst_pair['dataset']}")
print(f"Score: {worst_pair['score']}")
print("Parameters:")
print(format_params(worst_pair['params']))

# Esegui la strategia con i migliori e peggiori parametri trovati
print(f"\nBest pair transactions:\nPunteggio: {best_pair['score']}\nDataset: {best_pair['dataset']}\nParametri: {best_pair['params']}\nTransazioni:")
best_transactions = strategia_vendita(
    dati_da_usare[best_pair['dataset']],
    **{**best_pair['params'], **parametri_fissi}
)
display(best_transactions)

print(f"\nWorst pair transactions:\nPunteggio: {worst_pair['score']}\nDataset: {worst_pair['dataset']}\nParametri: {worst_pair['params']}\nTransazioni:")
worst_transactions = strategia_vendita(
    dati_da_usare[worst_pair['dataset']],
    **{**worst_pair['params'], **parametri_fissi}
)
display(worst_transactions)


Optimization completed.
Best average score: 1.6035375630992346, with parameters soglia_sopportabile = 0.15, vendita_rispetto_inizio = True, paracadute_variabile = True, sopportazione_variabile = True, pct_paracadute = 0.0, A = 0.0, B = 0.0

Best pair details:
Dataset: 0
Score: 1.6035375630992346
Parameters:
soglia_sopportabile = 0.15, vendita_rispetto_inizio = True, paracadute_variabile = True, sopportazione_variabile = True, pct_paracadute = 0.0, A = 0.0, B = 0.0

Worst pair details:
Dataset: 0
Score: 0.956837263185918
Parameters:
soglia_sopportabile = 0.9894036157615339, vendita_rispetto_inizio = False, paracadute_variabile = False, sopportazione_variabile = False, pct_paracadute = 0.3440341197898761, A = 11.313022300957627, B = 32.160441866801136

Best pair transactions:
Punteggio: 1.6035375630992346
Dataset: 0
Parametri: {'soglia_sopportabile': 0.15, 'vendita_rispetto_inizio': True, 'paracadute_variabile': True, 'sopportazione_variabile': True, 'pct_paracadute': 0.0, 'A': 0.0, 'B'

,valore_totale_portafoglio
0,731.090775
1,771.185338
2,764.744820
3,784.537608
4,807.589932
...,...
145,1104.342204
146,1104.342204
147,1104.342204
148,1104.342204



Worst pair transactions:
Punteggio: 0.956837263185918
Dataset: 0
Parametri: {'soglia_sopportabile': 0.9894036157615339, 'vendita_rispetto_inizio': False, 'paracadute_variabile': False, 'sopportazione_variabile': False, 'pct_paracadute': 0.3440341197898761, 'A': 11.313022300957627, 'B': 32.160441866801136}
Transazioni:


,valore_totale_portafoglio
0,731.090775
1,771.185338
2,764.744820
3,784.537608
4,807.589932
...,...
145,658.965400
146,658.965400
147,658.965400
148,658.965400
